In [0]:
Bunit=0
PeriodValuePrefix=PeriodValue[:1]

if PeriodValuePrefix in ('Y','9','H','Q','M'):
    PeriodValueInYHQM=1
else:
    PeriodValueInYHQM=0

returnResult="Success"
showlog=False

config = Configuration(engagement)
dbschema = get_tbl_nm('')[:-1]
catalogname = dbschema.split(".")[0]
engagementname = dbschema.split(".")[1]
dbschema=catalogname+"."+engagementname


In [0]:
from datetime import datetime , date
from dateutil.relativedelta import relativedelta
from databricks import sql

In [0]:
def sqlwarehouse_queryExecutor(queryString):
    vSQLEPHost=config.adb_host
    vSQLEPHttp=config.adb_path
    vSQLEPPId=config.adb_token
    with sql.connect(server_hostname=f"{vSQLEPHost}",http_path=f"{vSQLEPHttp}",access_token=f"{vSQLEPPId}") as connection:
        with connection.cursor() as cursor:
            for query in queryString.split(";"):
                if len(query.strip()) > 0:
                    if showlog:print("\n" + "[Info] Executing query: "+ query )
                    cursor.execute(query)

In [0]:

def read_mssql(tblName):
    try:
        df_sqldata=spark.read.format("sqlserver").option("host",config.sqlhost).option("port",1433).option("user",config.sqlusername).option("password",config.sqlpassword).option("database",config.sqldbName).option("dbtable", tblName).load()
        return df_sqldata
    except Exception as e:
        returnResult="Failure"
        # print("Error Occured While Fetching Data From MS SQL Server : "+ str(e))
        raise e
        dbutils.notebook.exit(returnResult)



In [0]:
df_entitymaster=read_mssql("master.entitymaster")
df_entitymaster=df_entitymaster.filter(df_entitymaster["entitymasterid"] == EntityID).select("fk_fiscalperiodid","fk_functionalcurrencyid", "entityname","fk_fiscalperiodid").first()

FK_FiscalPeriodID = df_entitymaster["fk_fiscalperiodid"]
SDCurrencyid = df_entitymaster["fk_functionalcurrencyid"]
EntitySDName = df_entitymaster["entityname"]
FinancialSDCycleID = df_entitymaster["fk_fiscalperiodid"]
if showlog:
    # print("FK_FiscalPeriodID:",FK_FiscalPeriodID)
    # print("SDCurrencyid:",SDCurrencyid)
    print("EntitySDName:",EntitySDName)
    # print("FinancialSDCycleID:",FinancialSDCycleID)

In [0]:

df_financialcycle=read_mssql("master.financialcycle")
df_financialcycle_f = df_financialcycle.filter(df_financialcycle["financialcycleid"] == FK_FiscalPeriodID).select("financialcycle","financialcycleid").first()
FiscalPeriod = df_financialcycle_f["financialcycle"]
FiscalPeriodID = df_financialcycle_f["financialcycleid"]

df_financialcycle_f1 = df_financialcycle.filter(df_financialcycle["financialcycleid"] == FiscalPeriodID).select("Q1Start").first()
startVal=df_financialcycle_f1["Q1Start"]

if showlog:
    print("FiscalPeriod : ",FiscalPeriod)
    # print("FiscalPeriodID : ",FiscalPeriodID)
    # print("startVal : ",startVal)



In [0]:
df_periodicaldates=read_mssql("master.periodicaldates")
df_periodicaldates.createOrReplaceTempView("v_periodicaldates")
df_categorydropdown=read_mssql("master.categorydropdown")
df_categorydropdown.createOrReplaceTempView("v_categorydropdown")
df_EntityPeriodDetails=read_mssql("master.EntityPeriodDetails")
df_EntityPeriodDetails.createOrReplaceTempView("v_EntityPeriodDetails")
df_PeriodicalDatesForExtensionPeriods=read_mssql("master.PeriodicalDatesForExtensionPeriod")
df_PeriodicalDatesForExtensionPeriods.createOrReplaceTempView("v_PeriodicalDatesForExtensionPeriod")

In [0]:
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_daterangeconverttoperiod_" + str(parameterID) + "")
spark.sql("CREATE TABLE "+dbschema+".temp_daterangeconverttoperiod_" + str(parameterID) + "(id INT,currentperiod VARCHAR(50),startdate date, enddate date,periodstart VARCHAR(50),monthdiff int, periodmonthvalue VARCHAR(50),tbperiodvalue VARCHAR(50))")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + "")
spark.sql("CREATE TABLE "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + "(id INT,cpid VARCHAR(10),currentperiod VARCHAR(50),startdate DATE,enddate DATE,periodstart VARCHAR(50),monthdiff INT,periodmonthvalue VARCHAR(50),tbperiodvalue VARCHAR(50),Year VARCHAR(4),periodvalue VARCHAR(50),customperiod VARCHAR(50),customdaterange INT,openperiodvalue VARCHAR(50),opencyclevalue VARCHAR(50))")


In [0]:
alldimcols= "DimType1ID_fk,DimType2ID_fk,DimType3ID_fk,DimType4ID_fk,DimType5ID_fk,DimType6ID_fk,DimType7ID_fk,DimType8ID_fk,DimType9ID_fk,DimType10ID_fk,DimType11ID_fk,DimType12ID_fk,DimType13ID_fk,DimType14ID_fk,DimType15ID_fk,DimType16ID_fk,DimType17ID_fk,DimType18ID_fk,DimType19ID_fk,DimType20ID_fk,DimType21ID_fk,DimType22ID_fk,DimType23ID_fk,DimType24ID_fk,DimType25ID_fk"

alldimcols_dt= "DimType1ID_fk INT,DimType2ID_fk INT,DimType3ID_fk INT,DimType4ID_fk INT,DimType5ID_fk INT,DimType6ID_fk INT,DimType7ID_fk INT,DimType8ID_fk INT,DimType9ID_fk INT,DimType10ID_fk INT,DimType11ID_fk INT,DimType12ID_fk INT,DimType13ID_fk INT,DimType14ID_fk INT,DimType15ID_fk INT,DimType16ID_fk INT,DimType17ID_fk INT,DimType18ID_fk INT,DimType19ID_fk INT,DimType20ID_fk INT,DimType21ID_fk INT,DimType22ID_fk INT,DimType23ID_fk INT,DimType24ID_fk INT,DimType25ID_fk INT"

In [0]:
if PeriodValueInYHQM == 0:

  spark.sql("INSERT INTO "+dbschema+".temp_daterangeconverttoperiod_" + str(parameterID) + " SELECT monotonically_increasing_id()+1,'"+str(PeriodValue)+"',to_date(substr('"+str(PeriodValue)+"',0,10), 'dd-MM-yyyy' ),to_date(substring('"+str(PeriodValue)+"',12,LEN('"+str(PeriodValue)+"')-1), 'dd-MM-yyyy'),"+str(startVal)+",0,'','' ")

  if len(ComparitivePeriodValue)>0:
    
    spark.sql("INSERT INTO "+dbschema+".temp_daterangeconverttoperiod_" + str(parameterID) + " SELECT monotonically_increasing_id()+2,col_period,to_date(substr(col_period,0,10), 'dd-MM-yyyy' ),to_date(substring(col_period,12,LEN(col_period)-1), 'dd-MM-yyyy'),"+str(startVal)+",0,'','' FROM (SELECT explode(split('"+str(ComparitivePeriodValue)+"', ',')) as col_period ) as temp")
  
  spark.sql("UPDATE "+dbschema+".temp_daterangeconverttoperiod_" + str(parameterID) + " SET periodmonthvalue= concat('M',(month(enddate)-2+12)%12+1) , monthdiff= Datediff(month, startdate, enddate) + 1 , tbperiodvalue=REPLACE(SUBSTR(enddate,0,7),'-')")

  ############### Populating temp_daterangecustomperiod Data ###############

  spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_daterangecustomperiod_" + str(parameterID) + "")

  spark.sql("CREATE TABLE "+dbschema+".temp_daterangecustomperiod_" + str(parameterID) + " as SELECT pt.*, year, Concat(value, '-' , opening, '-' ,closing) AS periodvalue,  CASE WHEN periodmonthvalue = 'M12' AND  monthdiff = 12 THEN 'Year' WHEN periodmonthvalue = 'M12' AND  monthdiff = 6 THEN 'H2' WHEN periodmonthvalue = 'M12' AND  monthdiff = 3 THEN 'Q4' WHEN periodmonthvalue = 'M3' AND  monthdiff = 3 THEN 'Q1' WHEN periodmonthvalue = 'M6' AND  monthdiff = 6 THEN 'H1' WHEN periodmonthvalue = 'M6' AND  monthdiff = 3 THEN 'Q2' WHEN periodMonthValue ='M9'  AND monthdiff=9  then '9M' WHEN periodmonthvalue = 'M9' AND  monthdiff = 3 THEN 'Q3' ELSE concat('C-',substr(opening,0,4),SUBSTRING(startdate,6,2),'-',closing)  END as customperiod,  0 as customdaterange  FROM v_periodicaldates pd JOIN  "+dbschema+".temp_daterangeconverttoperiod_" + str(parameterID) + "  pt ON  pd.closing=pt.tbperiodvalue AND substring( pd.period, 1, charindex(' ', pd.period)-1)  = pt.periodmonthvalue AND  financialcycleid="+str(FiscalPeriodID)+" ")

  spark.sql("UPDATE "+dbschema+".temp_daterangecustomperiod_" + str(parameterID) + " SET periodvalue = CASE WHEN LEFT(customperiod,1)='C' THEN periodvalue ELSE REPLACE(periodvalue,Substring( periodvalue, 1, Charindex('-', periodvalue)-1 ),customperiod) END, customdaterange=CASE WHEN LEFT(customperiod,1)='C' THEN 1 ELSE 0 END")
  
  ############### Populating temp_daterangetoperiod Data ###############

  spark.sql("INSERT INTO "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + " SELECT id, concat('CP',id-1) as cpid, currentperiod, startdate, enddate, periodstart, CASE WHEN LEFT(periodmonthvalue, 1) = 'Y' THEN 12 WHEN left(periodmonthvalue,1) = '9' THEN 9  WHEN LEFT(periodmonthvalue, 1) = 'H' THEN 6 WHEN LEFT(periodmonthvalue, 1) = 'Q' THEN 3 WHEN LEFT(periodmonthvalue, 1) = 'M' THEN 1 END, periodmonthvalue, tbperiodvalue, Year, periodvalue, customperiod, customdaterange, '', '' FROM  "+dbschema+".temp_daterangecustomperiod_" + str(parameterID) + " ")

  spark.sql("MERGE INTO "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + " ptb USING v_periodicaldates pd ON ptb.periodmonthvalue=pd.value and financialcycleid="+str(FiscalPeriodID)+" and  pd.year = ptb.year WHEN MATCHED THEN UPDATE SET openperiodvalue = REPLACE(SUBSTR(startdate,0,7),'-') , opencyclevalue = pd.opening")

  PeriodValue=""
  PeriodValue_df=spark.sql("SELECT periodvalue FROM "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + " WHERE ID=1 LIMIT 1")
  if PeriodValue_df.count()>0 : PeriodValue=PeriodValue_df.collect()[0][0]

In [0]:
PeriodValuePrefix=PeriodValue[:1]
PeriodType=PeriodValuePrefix
endperiod=PeriodValue[-6:]
NumericalValueofPeriodType=""
match PeriodType:
	case 'Y':  NumericalValueofPeriodType=-12
	case '9':  NumericalValueofPeriodType=-9
	case 'H':  NumericalValueofPeriodType=-6
	case 'Q':  NumericalValueofPeriodType=-3
	case 'M':  NumericalValueofPeriodType=-1

if PeriodValueInYHQM == 0:
    NumericalValueofPeriodType_df = spark.sql("SELECT monthdiff FROM "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + " WHERE ID=1 LIMIT 1")
    if NumericalValueofPeriodType_df.count() > 0 : NumericalValueofPeriodType=NumericalValueofPeriodType_df.collect()[0][0]

YearEndinDateFormat = datetime.strptime("01 "+FiscalPeriod[-3:]+" "+FinancialYear, "%d %b %Y").date()
PeriodEndInDateFormat  = datetime.strptime(endperiod+"01", "%Y%m%d").date()

CurrentPeriod=endperiod
CurrentPeriod_YTD=endperiod
PreviousPeriod=""

if PeriodValuePrefix != '9' :
	PreviousPeriod = (PeriodEndInDateFormat + relativedelta(months=NumericalValueofPeriodType)).strftime("%Y%m")
else:
	PreviousPeriod = (PeriodEndInDateFormat + relativedelta(years=-1)).strftime("%Y%m")

PrevPreviousPeriod = (PeriodEndInDateFormat + relativedelta(months=2*NumericalValueofPeriodType)).strftime("%Y%m")
PreviousYear = (YearEndinDateFormat + relativedelta(years=-1)).strftime("%Y%m")
PrevPreviousYear  = (YearEndinDateFormat + relativedelta(years=-2)).strftime("%Y%m")
PrevYearCurrPeriod = (PeriodEndInDateFormat + relativedelta(years=-1)).strftime("%Y%m")
PrevYearCurrPeriod_YTD =  (PeriodEndInDateFormat + relativedelta(years=-1)).strftime("%Y%m")
PrevYearPrevPeriod = (PeriodEndInDateFormat + relativedelta(months=-12 + NumericalValueofPeriodType)).strftime("%Y%m")
PPrevPreviousYear = (YearEndinDateFormat + relativedelta(years=-3)).strftime("%Y%m")
P4PreviousYear = (YearEndinDateFormat + relativedelta(years=-4)).strftime("%Y%m")

if showlog:
	print("PeriodValue                : " , PeriodValue)
	# print("PeriodValuePrefix          : " , PeriodValuePrefix)
	# print("PeriodType                 : " , PeriodType)
	# print("endperiod                  : " , endperiod)
	# print("NumericalValueofPeriodType : " , NumericalValueofPeriodType)
	# print("YearEndinDateFormat        : " , YearEndinDateFormat)
	# print("PeriodEndInDateFormat      : " , PeriodEndInDateFormat)
	# print("CurrentPeriod              : " , CurrentPeriod)
	# print("CurrentPeriod_YTD          : " , CurrentPeriod_YTD)
	# print("PreviousPeriod             : " , PreviousPeriod)
	# print("PrevPreviousPeriod         : " , PrevPreviousPeriod)
	# print("PreviousYear               : " , PreviousYear)
	# print("PrevPreviousYear           : " , PrevPreviousYear)
	# print("PrevYearCurrPeriod         : " , PrevYearCurrPeriod)
	# print("PrevYearCurrPeriod_YTD     : " , PrevYearCurrPeriod_YTD)
	# print("PrevYearPrevPeriod         : " , PrevYearPrevPeriod)
	# print("P4PreviousYear             : " , P4PreviousYear)

In [0]:
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_complist_" + str(parameterID) + "")
spark.sql("CREATE TABLE "+dbschema+".temp_complist_" + str(parameterID) + "(compperiod VARCHAR(50))")

if PeriodValueInYHQM == 1:
	spark.sql("INSERT INTO "+dbschema+".temp_complist_" + str(parameterID) + " SELECT explode(split('"+str(ComparitivePeriodValue)+"', ',')) as col_period")
else:
	spark.sql("INSERT INTO "+dbschema+".temp_complist_" + str(parameterID) + " SELECT CONCAT(CASE WHEN customdaterange = 1 THEN periodmonthvalue ELSE customperiod  END,'-',openperiodvalue,'-',tbperiodvalue)  FROM "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + " WHERE ID<>1 ")
 
countofcomp = ""
countofcomp_df=spark.sql("SELECT count(*) as complist_count FROM  "+dbschema+".temp_complist_" + str(parameterID) + "")
if countofcomp_df.count() > 0 : countofcomp=countofcomp_df.collect()[0][0]

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_compperioddetails_" + str(parameterID) + "")

spark.sql("CREATE TABLE "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + "(id INT, compperiodname VARCHAR(500), compperiodvalue VARCHAR(50), comptype VARCHAR(10), compperiod VARCHAR(6), Value VARCHAR(10)) ")
spark.sql("CREATE TABLE "+dbschema+".temp_compperioddetails_" + str(parameterID) + "(comptype VARCHAR(5), alias VARCHAR(500))")
ComparitiveSelectionEndPeriod=""
CompYearEndinDateFormat =""
CompPeriodEndInDateFormat =""
gapincompperiod =""
if ComparitivePeriodValue is not None and len(ComparitivePeriodValue) > 0:
	if countofcomp==1:
		spark.sql("INSERT INTO "+dbschema+".temp_compperioddetails_" + str(parameterID) + " VALUES ('CP1', 'PreviousPeriod'),('CP1', 'PrevYearCurrPeriod_YTD'),('CP1', 'PrevYearCurrPeriod'),('CP1', 'PreviousYear')")
	elif countofcomp==2:
		spark.sql("INSERT INTO "+dbschema+".temp_compperioddetails_" + str(parameterID) + " VALUES ('CP1', 'PreviousPeriod'),('CP1', 'PreviousYear'),('CP2', 'PrevPreviousYear')")
	elif countofcomp==3:
		spark.sql("INSERT INTO "+dbschema+".temp_compperioddetails_" + str(parameterID) + " VALUES ('CP1', 'PreviousPeriod'), ('CP1', 'PreviousYear'), ('CP2', 'PrevPreviousYear'), ('CP3', 'PPrevPreviousYear')")
	elif countofcomp==4:
		spark.sql("INSERT INTO "+dbschema+".temp_compperioddetails_" + str(parameterID) + " VALUES ('CP1', 'PreviousPeriod'), ('CP1', 'PreviousYear'), ('CP2', 'PrevPreviousYear'), ('CP3', 'PPrevPreviousYear'), ('CP4','P4PreviousYear')")
	elif countofcomp==5:
		spark.sql("INSERT INTO "+dbschema+".temp_compperioddetails_" + str(parameterID) + " VALUES ('CP1', 'PreviousPeriod'), ('CP1', 'PreviousYear'), ('CP2', 'PrevPreviousYear'), ('CP3', 'PPrevPreviousYear'), ('CP4','P4PreviousYear'), ('CP5','P5PreviousYear')")
	elif countofcomp==6:
		spark.sql("INSERT INTO "+dbschema+".temp_compperioddetails_" + str(parameterID) + " VALUES ('CP1', 'PreviousPeriod'), ('CP1', 'PreviousYear'), ('CP2', 'PrevPreviousYear'), ('CP3', 'PPrevPreviousYear'), ('CP4','P4PreviousYear'), ('CP5','P5PreviousYear'), ('CP6','P6PreviousYear') ")
	elif countofcomp==7:
		spark.sql("INSERT INTO "+dbschema+".temp_compperioddetails_" + str(parameterID) + " VALUES ('CP1', 'PreviousPeriod'), ('CP1', 'PreviousYear'), ('CP2', 'PrevPreviousYear'), ('CP3', 'PPrevPreviousYear'), ('CP4','P4PreviousYear'), ('CP5','P5PreviousYear'), ('CP6','P6PreviousYear'), ('CP7','P7PreviousYear') ")
	elif countofcomp==8:
		spark.sql("INSERT INTO "+dbschema+".temp_compperioddetails_" + str(parameterID) + " VALUES ('CP1', 'PreviousPeriod'), ('CP1', 'PreviousYear'), ('CP2', 'PrevPreviousYear'), ('CP3', 'PPrevPreviousYear'), ('CP4','P4PreviousYear'), ('CP5','P5PreviousYear'), ('CP6','P6PreviousYear'), ('CP7','P7PreviousYear'), ('CP8','P8PreviousYear') ")

	spark.sql(" INSERT INTO "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + " SELECT monotonically_increasing_id()+1 as id,alias, CASE WHEN "+str(PeriodValueInYHQM)+"=1 THEN compperiodvalue ELSE CONCAT(CASE WHEN customdaterange = 1 THEN periodmonthvalue ELSE customperiod END,'-',openperiodvalue,'-', tbperiodvalue) END as compperiodvalue,pt.comptype, CASE WHEN "+str(PeriodValueInYHQM)+"=1 THEN substring(compperiodvalue,-6)  ELSE  tbperiodvalue END as compperiod,'' FROM (SELECT concat('CP',monotonically_increasing_id()+1) as comptype,t.compperiodvalue FROM ( SELECT explode(split('"+ComparitivePeriodValue+"', ',')) as compperiodvalue) as t ) as ct JOIN "+dbschema+".temp_compperioddetails_" + str(parameterID) + " pt ON ct.comptype = pt.comptype LEFT JOIN "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + " dp on ct.comptype=dp.cpid")

	spark.sql("UPDATE "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + " SET Value=substring(compperiodvalue,0, charindex('-', compperiodvalue)-1)")
	CompFinancialyear=""
	CompFinancialyear_df = spark.sql("SELECT CASE WHEN '"+FiscalPeriod+"' = 'Jan-Dec' THEN substring(substring(compperiodvalue,-6),0,4) ELSE CAST(Substring( compperiodvalue,Charindex('-', compperiodvalue)+ 1, 4 ) + 1 as INT) END FROM "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + " WHERE id=1  ")
	if CompFinancialyear_df.count() > 0 : CompFinancialyear=CompFinancialyear_df.collect()[0][0]
    
    
	ComparitiveSelectionEndPeriod_df = spark.sql("SELECT substring(compperiodvalue,-6) FROM "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + " WHERE id=1 ")
	if ComparitiveSelectionEndPeriod_df.count() > 0 : ComparitiveSelectionEndPeriod=ComparitiveSelectionEndPeriod_df.collect()[0][0]
	
	CompYearEndinDateFormat = datetime.strptime("01 "+FiscalPeriod[-3:]+" "+CompFinancialyear, "%d %b %Y").date()
	CompPeriodEndInDateFormat = datetime.strptime(ComparitiveSelectionEndPeriod+"01", "%Y%m%d").date()
	gapincompperiod = relativedelta(CompYearEndinDateFormat, YearEndinDateFormat).years

if showlog:
    print("ComparitiveSelectionEndPeriod          : " , ComparitiveSelectionEndPeriod)
    # print("CompYearEndinDateFormat                : " , CompYearEndinDateFormat)
    # print("CompPeriodEndInDateFormat              : " , CompPeriodEndInDateFormat)
    # print("gapincompperiod                        : " , gapincompperiod)

In [0]:
countofcomp_1=0
countofcomp_1_df=spark.sql("SELECT count(*) as complist_count FROM  "+dbschema+".temp_complist_" + str(parameterID) + "")
if countofcomp_1_df.count() > 0 : countofcomp_1=countofcomp_1_df.collect()[0][0]

if countofcomp_1 == 1 and PeriodValueInYHQM == 1 :
    spark.sql("TRUNCATE TABLE "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + "")
    if ComparitivePeriodValue is not None and len(ComparitivePeriodValue) > 0:
        PreviousPeriod = ComparitiveSelectionEndPeriod
        PrevPreviousPeriod = (CompPeriodEndInDateFormat + relativedelta(months=NumericalValueofPeriodType)).strftime("%Y%m%d")
        PreviousYear= (CompYearEndinDateFormat + relativedelta(years=0)).strftime("%Y%m%d")
        PrevPreviousYear= (CompYearEndinDateFormat + relativedelta(years=-1)).strftime("%Y%m%d")
        PrevYearCurrPeriod = (CompPeriodEndInDateFormat + relativedelta(years=0)).strftime("%Y%m%d")
        PrevYearCurrPeriod_YTD = (CompPeriodEndInDateFormat + relativedelta(years=0)).strftime("%Y%m%d")
        PrevYearPrevPeriod=(CompPeriodEndInDateFormat + relativedelta(months=NumericalValueofPeriodType)).strftime("%Y%m%d")
        PPrevPreviousYear= (CompYearEndinDateFormat + relativedelta(years=-2)).strftime("%Y%m%d")
        P4PreviousYear= (CompYearEndinDateFormat + relativedelta(years=-3)).strftime("%Y%m%d")


spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_tmpdates_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_dates_" + str(parameterID) + "")
spark.sql("CREATE TABLE "+dbschema+".temp_tmpdates_" + str(parameterID) + "(periodtype VARCHAR(50),currentvalue VARCHAR(6), legacyperiod VARCHAR(20)) ")
spark.sql("CREATE TABLE "+dbschema+".temp_dates_" + str(parameterID) + "(periodtype VARCHAR(50),currentvalue VARCHAR(6), legacyperiod VARCHAR(20)) ")

if (countofcomp == 0 or countofcomp == 1) and PeriodValueInYHQM == 1 :
    spark.sql("INSERT INTO "+dbschema+".temp_tmpdates_" + str(parameterID) + " VALUES ('currentPeriod',LEFT('"+CurrentPeriod+"',6), ''),('currentPeriod_YTD',LEFT('"+CurrentPeriod_YTD+"',6),''),('PreviousPeriod',LEFT('"+PreviousPeriod+"',6),''),('PreviousYear', LEFT('"+PreviousYear+"',6),''),('PrevYearCurrPeriod', LEFT('"+PrevYearCurrPeriod+"',6),''), ('PrevYearCurrPeriod_YTD', LEFT('"+PrevYearCurrPeriod_YTD+"',6),'') ")
    spark.sql("DELETE FROM "+dbschema+".temp_tmpdates_" + str(parameterID) + " WHERE periodtype IN (SELECT compperiodname DISTINCT FROM "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + ")")
    spark.sql("INSERT INTO "+dbschema+".temp_dates_" + str(parameterID) + " SELECT periodtype,currentvalue,legacyperiod FROM "+dbschema+".temp_tmpdates_" + str(parameterID) + " UNION SELECT  compperiodname as periodtype,compperiod as currentvalue,'' as legacyperiod FROM "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + "")
    spark.sql("UPDATE "+dbschema+".temp_dates_" + str(parameterID) + " SET legacyperiod = CASE WHEN "+str(PeriodValueInYHQM)+" = 1 THEN CASE WHEN PeriodType LIKE '%Year' THEN concat(currentvalue,'-12') ELSE CASE '"+str(PeriodValuePrefix)+"'  WHEN 'Y' THEN concat(currentvalue,'-12') WHEN '9' THEN concat(currentvalue,'-09') WHEN 'H' THEN concat(currentvalue,'-06') WHEN 'Q' THEN concat(currentvalue,'-03') WHEN 'M' THEN concat(currentvalue,'-01') END END END" )
    spark.sql("MERGE INTO "+dbschema+".temp_dates_" + str(parameterID) + " d USING "+dbschema+".temp_complist_" + str(parameterID) + " c ON d.currentvalue = RIGHT(c.compperiod, 6) WHEN MATCHED THEN UPDATE SET legacyperiod = CASE LEFT( Substring( compperiod, 0, Charindex('-', compperiod)-1 ), 1 ) WHEN 'Y' THEN concat(currentvalue,'-12') WHEN '9' THEN concat(currentvalue,'-09') WHEN 'H' THEN concat(currentvalue,'-06') WHEN 'Q' THEN concat(currentvalue,'-03') WHEN 'M' THEN concat(currentvalue,'-01') END")

    NumericalPeriodTypeValue = abs(NumericalValueofPeriodType)

    spark.sql("UPDATE "+dbschema+".temp_dates_" + str(parameterID) + " SET legacyperiod = concat(currentvalue,'-',CASE WHEN len("+str(NumericalPeriodTypeValue)+")=1 THEN concat('0','"+str(NumericalPeriodTypeValue)+"') ELSE '"+str(NumericalPeriodTypeValue)+"'END)  WHERE legacyperiod IS NULL" )

else:
    spark.sql("INSERT INTO "+dbschema+".temp_tmpdates_" + str(parameterID) + " VALUES ('currentPeriod','"+CurrentPeriod+"', '')")
    spark.sql("INSERT INTO "+dbschema+".temp_dates_" + str(parameterID) + " SELECT periodtype,currentvalue,CASE '"+PeriodValuePrefix+"' WHEN  'Y' THEN concat(currentvalue ,'-12') WHEN  '9' THEN concat(currentvalue,'-09') WHEN  'H' THEN concat(currentvalue,'-06') WHEN  'Q' THEN concat(currentvalue,'-03') WHEN  'M' THEN concat(currentvalue, '-01') END FROM "+dbschema+".temp_tmpdates_" + str(parameterID) + " UNION SELECT  compperiodname as periodtype,compperiod as currentvalue,CASE LEFT(Value,1) WHEN  'Y' THEN concat(compperiod ,'-12') WHEN  '9' THEN concat(compperiod,'-09') WHEN  'H' THEN concat(compperiod,'-06') WHEN  'Q' THEN concat(compperiod,'-03') WHEN  'M' THEN concat(compperiod, '-01') END legacyperiod FROM "+dbschema+".temp_usergivenmulticomp_" + str(parameterID) + "")

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_daterangetoperiodtype_" + str(parameterID) + "")
spark.sql("CREATE TABLE  "+dbschema+".temp_daterangetoperiodtype_" + str(parameterID) + "  as SELECT DISTINCT dp.*,CASE WHEN alias IS NULL THEN 'CurrentPeriod' ELSE cp.alias END periodtype FROM "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + " dp LEFT JOIN "+dbschema+".temp_compperioddetails_" + str(parameterID) + " cp ON dp.cpid = cp.comptype")


In [0]:

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_tempcategory_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_legacysddates_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_ladjjfinal_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_castingsdadjustment_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_castingsdadjustment1_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_allperiodftpscoa_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_standaloneadj_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_Trialbalance_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_legacyadjustment_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_TB_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_SCOAMappingTable_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_SCOAMappingTable1_" + str(parameterID) + "")

CategorySDName_df=spark.sql("SELECT categoryname FROM v_categorydropdown where categoryid="+str(CategoryID)+"")
if CategorySDName_df.count() > 0 : CategorySDName=CategorySDName_df.collect()[0][0]

spark.sql("CREATE TABLE  "+dbschema+".temp_tempcategory_" + str(parameterID) + " as SELECT categoryid from v_categorydropdown where categoryname='"+str(CategorySDName)+"' and entityid="+str(EntityID)+" and accountingstandardid="+str(AccountingStandardID)+"")

############### SCOA Mapping Data Creation ###############

spark.sql("CREATE TABLE "+dbschema+".temp_allperiodftpscoa_" + str(parameterID) + "(SCOAID int,Member varchar(1000),Path varchar(8000),Column27 varchar(1000),AccountSubType varchar(500),UltimateParent varchar(200) )")
spark.sql("INSERT INTO "+dbschema+".temp_allperiodftpscoa_" + str(parameterID) + "(SCOAID) SELECT SCOAID FROM "+dbschema+".scoa WHERE mcoayear="+str(FinancialYear)+" and accountingtype="+str(AccountingStandardID)+"")
spark.sql("MERGE INTO "+dbschema+".temp_allperiodftpscoa_" + str(parameterID) + " ap USING "+dbschema+".scoa sc ON ap.SCOAID = sc.SCOAID WHEN MATCHED THEN UPDATE SET ap.Path=sc.Path, ap.Member = sc.Member,ap.column27 = sc.column27,ap.AccountSubType=sc.AccountSubType,ap.UltimateParent=sc.UltimateParent")

In [0]:

temp_drilldown_tbl = get_tbl_nm('multiperiod_drilldown_'+str(parameterID)).split('.')[-1]
temp_staging_tbl = get_tbl_nm('multiperiod_staging_'+str(parameterID)).split('.')[-1]

In [0]:
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_legacysddates_" + str(parameterID) + "")

spark.sql("CREATE TABLE "+dbschema+".temp_legacysddates_" + str(parameterID) + "( id INT, periodtype VARCHAR(500), Value VARCHAR(5), pcurrentvalue VARCHAR(50), currentvalue VARCHAR(50), legacyperiod VARCHAR(20), categoryid INT, financialyear INT, openingperiodvalue VARCHAR(20), castingperiod VARCHAR(20), numericvalueofperiod INT, ftpperiod INT, periodformat VARCHAR(20), p VARCHAR(20), ppp VARCHAR(20), py VARCHAR(20), pypp VARCHAR(20), ppy VARCHAR(20), p3y VARCHAR(20), p4y VARCHAR(20) )")

spark.sql("INSERT INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + "(id,periodtype,currentvalue,pcurrentvalue,legacyperiod,categoryid,castingperiod) SELECT monotonically_increasing_id()+1,periodtype,currentvalue,currentvalue,legacyperiod,0,12 from "+dbschema+".temp_dates_" + str(parameterID) + "")

spark.sql("merge into "+dbschema+".temp_legacysddates_" + str(parameterID) + " s using (select ld.id,dp.customdaterange from  "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld left join "+dbschema+".temp_daterangetoperiodtype_" + str(parameterID) + " dp on ld.currentvalue=dp.tbperiodvalue and ld.periodtype=dp.periodtype) t on s.id=t.id when matched then update set s.periodformat=case when t.customdaterange<>1 or t.customdaterange is null then case right(s.legacyperiod,2) WHEN '12' THEN 'Yearly' WHEN '06' THEN 'Halfyearly' WHEN '03' THEN 'Quarterly' WHEN '01' THEN 'Monthly' WHEN '09' THEN '9 Months' ELSE 'Monthly' END WHEN t.customdaterange = 1 THEN 'Monthly' END")

spark.sql("merge into "+dbschema+".temp_legacysddates_" + str(parameterID) + " s using (select ld.id,pd.Value,pd.year,opening  from v_periodicaldates pd inner join "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld on pd.closing =ld.currentvalue left join "+dbschema+".temp_daterangetoperiodtype_" + str(parameterID) + " dp on ld.currentvalue=dp.tbperiodvalue and ld.periodtype=dp.periodtype  where financialcycleid = 2 and Type = periodformat ) as t on s.id=t.id when matched then update set s.Value=t.Value ,  s.financialyear=t.year , s.openingperiodvalue= case when s.openingperiodvalue is not null then s.openingperiodvalue else t.opening end ")

spark.sql("merge into "+dbschema+".temp_legacysddates_" + str(parameterID) + " s using (select ld.id,monthdiff,openperiodvalue from  "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld left join "+dbschema+".temp_daterangetoperiodtype_" + str(parameterID) + " dp on ld.currentvalue=dp.tbperiodvalue and ld.periodtype=dp.periodtype) t on s.id=t.id when matched then update set numericvalueofperiod = CASE WHEN "+str(PeriodValueInYHQM)+" = 1 THEN CASE LEFT(Value, 1) WHEN 'Y' THEN -12 WHEN '9' THEN -9 WHEN 'H' THEN -6 WHEN 'Q' THEN -3 WHEN 'M' THEN -1 END WHEN "+str(PeriodValueInYHQM)+" = 0 THEN CASE WHEN s.PeriodType LIKE '%Year' AND openperiodvalue IS NULL THEN -12 ELSE -monthdiff END END")

numericvalueofperiod=""
numericvalueofperiod_df=spark.sql("SELECT -monthdiff FROM "+dbschema+".temp_daterangetoperiod_" + str(parameterID) + " where id=1 ")
if numericvalueofperiod_df.count()>0 :numericvalueofperiod=numericvalueofperiod_df.collect()[0][0]
spark.sql("update "+dbschema+".temp_legacysddates_" + str(parameterID) + " set numericvalueofperiod= CASE WHEN length('"+str(numericvalueofperiod)+"')>0 THEN numericvalueofperiod ELSE NULL END where numericvalueofperiod is null ")

spark.sql("UPDATE "+dbschema+".temp_legacysddates_" + str(parameterID) + " SET castingperiod = CONCAT(Value, '-', openingperiodvalue, '-', currentvalue), FTPPeriod = CASE WHEN "+str(PeriodValueInYHQM)+" = 1 THEN CASE LEFT(Value, 1) WHEN 'Y' THEN 5 WHEN '9' THEN 4 WHEN 'H' THEN 3 WHEN 'Q' THEN 2 WHEN 'M' THEN 1 ELSE 1 END WHEN "+str(PeriodValueInYHQM)+" = 0 THEN 1 END,p=CONCAT(LEFT(DATE_ADD(MONTH,numericvalueofperiod,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),4),RIGHT(LEFT(DATE_ADD(MONTH,numericvalueofperiod,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),7),2)), ppp=CONCAT(LEFT(DATE_ADD(MONTH,2*numericvalueofperiod,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),4),RIGHT(LEFT(DATE_ADD(MONTH,2*numericvalueofperiod,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),7),2)), pypp=CONCAT(LEFT(DATE_ADD(MONTH, -12 + numericvalueofperiod,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),4),RIGHT(LEFT(DATE_ADD(MONTH, -12 + numericvalueofperiod,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),7),2)), py=CONCAT(LEFT(DATE_ADD(YEAR,-1,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),4),RIGHT(LEFT(DATE_ADD(YEAR,-1,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),7),2)), ppy=CONCAT(LEFT(DATE_ADD(YEAR,-2,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),4),RIGHT(LEFT(DATE_ADD(YEAR,-2,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),7),2)), p3y=CONCAT(LEFT(DATE_ADD(YEAR,-3,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),4),RIGHT(LEFT(DATE_ADD(YEAR,-3,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),7),2)), p4y=CONCAT(LEFT(DATE_ADD(YEAR,-4,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),4),RIGHT(LEFT(DATE_ADD(YEAR,-4,to_date(CONCAT('01-',RIGHT(currentvalue, 2),'-',LEFT(currentvalue, 4)),'dd-MM-yyyy')),7),2)) ") 


In [0]:
period_value_ld_df = spark.sql("SELECT SUBSTRING(legacyperiod, 1, 6) as period_value_ld from "+dbschema+".temp_legacysddates_" + str(parameterID) + "")
period_value_ld_list = [row.period_value_ld for row in period_value_ld_df.collect()]
period_value_ld = ",".join(["'" + value + "'" for value in period_value_ld_list])

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_standaloneadj_" + str(parameterID) + "")
spark.sql("CREATE TABLE "+dbschema+".temp_standaloneadj_" + str(parameterID) + " TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported') as SELECT * FROM "+dbschema+".adjustments where EntityID_fk="+str(EntityID)+" and AccountingStdID_fk="+str(AccountingStandardID)+" and CategoryID_fk="+str(CategoryID)+" and IsActive = true")

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_trialbalance_" + str(parameterID) + "")
spark.sql("CREATE TABLE "+dbschema+".temp_trialbalance_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') AS SELECT * FROM "+dbschema+".trialbalance WHERE EntityID_fk="+str(EntityID)+" and AccountingStdID_fk="+str(AccountingStandardID)+" and SUBSTRING(Period,1,6) in (select closing from v_periodicaldates where financialcycleid="+str(FiscalPeriodID)+")  and IsActive = true and SUBSTRING(Period,1,6) IN ("+period_value_ld+") ")

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_castingsdadjustment_" + str(parameterID) + "")
spark.sql("CREATE TABLE "+dbschema+".temp_castingsdadjustment_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') AS SELECT * FROM "+dbschema+".CastingAdjustmentTable WHERE fk_entityid = "+str(EntityID)+" AND fk_accountingstandardid = "+str(AccountingStandardID)+"")

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_legacyadjustment_" + str(parameterID) + "")
spark.sql("CREATE TABLE "+dbschema+".temp_legacyadjustment_" + str(parameterID) + " TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported') as SELECT * FROM  "+dbschema+".LegacyAdjustment WHERE  EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+"  and IsActive = true")

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_LegRecords_" + str(parameterID) + "")

spark.sql("CREATE TABLE IF NOT EXISTS "+dbschema+".temp_t_legacyadjustmentsdrilldown_" + str(parameterID) + " (LegacyAdjustmentID INT,AccountingStdID_fk INT,Period STRING,EntityID_Fk INT,JournalNumber INT,JournalDate TIMESTAMP,UserGLCode STRING,Member STRING,DebitCredit STRING,CurrencyID_Fk INT,Amount DECIMAL(20,5),JournalType STRING,Narration STRING,CreatedDate TIMESTAMP,CreatedBy STRING,ModifiedDate TIMESTAMP,ModifiedBy STRING,BUID_fk INT,IsImported BOOLEAN,CategoryID INT,IsMappedGLCode BOOLEAN DEFAULT TRUE,EndPeriod STRING,NotApplicable STRING,DimType1ID_fk INT,DimType2ID_fk INT,DimType3ID_fk INT,DimType4ID_fk INT,DimType5ID_fk INT,DimType6ID_fk INT,DimType7ID_fk INT,DimType8ID_fk INT,DimType9ID_fk INT,DimType10ID_fk INT,DimType11ID_fk INT,DimType12ID_fk INT,DimType13ID_fk INT,DimType14ID_fk INT,DimType15ID_fk INT,DimType16ID_fk INT,DimType17ID_fk INT,DimType18ID_fk INT,DimType19ID_fk INT,DimType20ID_fk INT,DimType21ID_fk INT,DimType22ID_fk INT,DimType23ID_fk INT,DimType24ID_fk INT,DimType25ID_fk INT)TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported')")

spark.sql("CREATE TABLE IF NOT EXISTS "+dbschema+".temp_LegRecords_" + str(parameterID) + " (LegacyAdjustmentID INT,AccountingStdID_fk INT,Period STRING,EntityID_Fk INT,JournalNumber INT,JournalDate TIMESTAMP,UserGLCode STRING,Member STRING,DebitCredit STRING,CurrencyID_Fk INT,Amount DECIMAL(20,5),JournalType STRING,Narration STRING,CreatedDate TIMESTAMP,CreatedBy STRING,ModifiedDate TIMESTAMP,ModifiedBy STRING,BUID_fk INT,IsImported BOOLEAN,CategoryID INT,IsMappedGLCode BOOLEAN DEFAULT TRUE,EndPeriod STRING,NotApplicable STRING,DimType1ID_fk INT,DimType2ID_fk INT,DimType3ID_fk INT,DimType4ID_fk INT,DimType5ID_fk INT,DimType6ID_fk INT,DimType7ID_fk INT,DimType8ID_fk INT,DimType9ID_fk INT,DimType10ID_fk INT,DimType11ID_fk INT,DimType12ID_fk INT,DimType13ID_fk INT,DimType14ID_fk INT,DimType15ID_fk INT,DimType16ID_fk INT,DimType17ID_fk INT,DimType18ID_fk INT,DimType19ID_fk INT,DimType20ID_fk INT,DimType21ID_fk INT,DimType22ID_fk INT,DimType23ID_fk INT,DimType24ID_fk INT,DimType25ID_fk INT)TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported')")

spark.sql("TRUNCATE TABLE "+dbschema+".temp_t_legacyadjustmentsdrilldown_" + str(parameterID) + "")

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_tempLegcat_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_tmpLegacywithoutendperiod_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_tmplegacywithoutNA_" + str(parameterID) + "")
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_tempLegacy_" + str(parameterID) + "")

spark.sql("CREATE TABLE "+dbschema+".temp_tempLegcat_" + str(parameterID) + " AS SELECT explode(split('"+str(CategoryID)+"', ',')) as categorid  ")

spark.sql("CREATE TABLE "+dbschema+".temp_tmpLegacywithoutendperiod_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') AS SELECT LegacyAdjustmentID,AccountingStdID_fk,Period,EntityID_fk,JournalNumber,JournalDate,UserGLCode,DebitCredit,CurrencyID_fk,Amount,JournalType,Narration,CreatedDate,CreatedBy,ModifiedDate,ModifiedBy,BUID_fk,IsImported,CategoryID,IsMappedGLCode,EndPeriod,NotApplicable, "+alldimcols+" FROM "+dbschema+".temp_legacyadjustment_" + str(parameterID) + " WHERE BUID_fk='"+str(Bunit)+"' and CategoryID in (select categoryid from "+dbschema+".temp_tempLegcat_" + str(parameterID) + " ) and EntityID_fk = "+str(EntityID)+" and AccountingStdID_fk = "+str(AccountingStandardID)+" and NVL(EndPeriod,999999-99) > Period ")


spark.sql("CREATE TABLE "+dbschema+".temp_tmplegacywithoutNA_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') SELECT *,case when Period = value then 0 when (NotApplicable is not null and NotApplicable !='' and value='') then 0 else 1 end IsmappedGLUpdatedCode FROM (SELECT *,explode(split(NVL(NotApplicable,''), ',')) as value  FROM "+dbschema+".temp_tmpLegacywithoutendperiod_" + str(parameterID) + ") as t")


spark.sql("CREATE TABLE "+dbschema+".temp_tempLegacy_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') AS SELECT DISTINCT LegacyAdjustmentID,AccountingStdID_fk,Period,EntityID_fk,JournalNumber,JournalDate,UserGLCode,DebitCredit,CurrencyID_fk,Amount, JournalType,Narration,CreatedDate,CreatedBy,ModifiedDate,ModifiedBy,BUID_fk,IsImported,CategoryID,IsMappedGLCode,EndPeriod,NotApplicable, "+alldimcols+" FROM "+dbschema+".temp_tmplegacywithoutNA_" + str(parameterID) + " WHERE IsmappedGLUpdatedCode=1 and period not in (select distinct value from   "+dbschema+".temp_tmplegacywithoutNA_" + str(parameterID) + " )")

spark.sql("INSERT INTO  "+dbschema+".temp_t_legacyadjustmentsdrilldown_" + str(parameterID) + " (LegacyAdjustmentID, AccountingStdID_fk, Period, EntityID_fk, JournalNumber, JournalDate, UserGLCode, DebitCredit, CurrencyID_fk, Amount, JournalType, Narration , CreatedDate, CreatedBy, ModifiedDate, ModifiedBy, BUID_fk, IsImported, CategoryID, IsMappedGLCode, EndPeriod, NotApplicable, "+alldimcols+" ) select LegacyAdjustmentID, AccountingStdID_fk, Period, EntityID_fk, JournalNumber, JournalDate, UserGLCode, DebitCredit, CurrencyID_fk, Amount, JournalType, Narration , CreatedDate, CreatedBy, ModifiedDate, ModifiedBy, BUID_fk, IsImported, CategoryID, IsMappedGLCode, EndPeriod, NotApplicable, "+alldimcols+" from "+dbschema+".temp_tempLegacy_" + str(parameterID) + " ")

spark.sql("INSERT INTO  "+dbschema+".temp_LegRecords_" + str(parameterID) + " SELECT *  from "+dbschema+".temp_t_legacyadjustmentsdrilldown_" + str(parameterID) + " ")

In [0]:
# Combaing TB , ADJ and Leagcy ADJ

########### Action Item : UserGLDescription is not avaliable in LegRecords and Unit is not avaliable -- Businnes Unit in Dimension

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_tb_" + str(parameterID) + "")

spark.sql("CREATE TABLE "+dbschema+".temp_tb_" + str(parameterID) + " AS SELECT DISTINCT UserGLCode,UserGLDescription,EntityID_fk,0 as unit FROM (SELECT UserGLCode,UserGLDescription,EntityID_fk,0 as unit FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " UNION ALL SELECT UserGLcode, NULL as UserGLDescription, EntityID_fk, 0 as unit FROM "+dbschema+".temp_standaloneadj_" + str(parameterID) + "  UNION ALL SELECT UserGLCode, NULL as UserGLDescription, EntityID_fk, 0 as unit FROM "+dbschema+".temp_LegRecords_" + str(parameterID) + ") as t ")

yearperiod = spark.sql("SELECT closing FROM v_periodicaldates WHERE financialcycleid = '"+str(FiscalPeriodID)+"'  AND year = '"+str(FinancialYear)+"' AND Type = 'Yearly'").collect()[0][0]

In [0]:
spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_SCOAMappingTable1_" + str(parameterID) + "")

spark.sql("CREATE TABLE "+dbschema+".temp_SCOAMappingTable1_" + str(parameterID) + " (EntityId int ,Period varchar(12) ,Accountingtype int ,EYGLCode varchar(500) ,UserGLCode varchar(500) ,CategoryID int ,UltimateParent varchar(100) ,Unit varchar(100) ,CreatedBy varchar(100) ,ModifiedBy varchar(100) ,CreatedDate varchar(100) ,ModifiedDate varchar(100) ,PresentationLayer varchar(100) ,UserGLDescription varchar(500) ) ")

spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_SCOAMappingTable_" + str(parameterID) + "")

spark.sql("CREATE TABLE "+dbschema+".temp_SCOAMappingTable_" + str(parameterID) + "(EntityId int ,Period varchar(12) ,Accountingtype int ,EYGLCode varchar(500) ,UserGLCode varchar(500) ,CategoryID int ,UltimateParent varchar(100) ,Unit varchar(100) ,CreatedBy varchar(100) ,ModifiedBy varchar(100) ,CreatedDate varchar(100) ,ModifiedDate varchar(100) ,PresentationLayer varchar(100) ,UserGLDescription varchar(500) )")

yearperiod=""
yearperiod_df = spark.sql("SELECT closing FROM v_periodicaldates WHERE financialcycleid = '"+str(FiscalPeriodID)+"'  AND year = '"+str(FinancialYear)+"' AND Type = 'Yearly'")
if yearperiod_df.count() > 0 : yearperiod=yearperiod_df.collect()[0][0]


IsAutomapped=""
IsAutomapped_df=spark.sql("SELECT IsAutomapped FROM v_categorydropdown where categoryid="+str(CategoryID)+"")
if IsAutomapped_df.count() > 0 : IsAutomapped=IsAutomapped_df.collect()[0][0]

if IsAutomapped==1:
  spark.sql("INSERT INTO "+dbschema+".temp_SCOAMappingTable1_" + str(parameterID) + " SELECT tb.EntityID_fk AS EntityId, concat('"+str(yearperiod)+"', '-12') AS Period, '"+str(AccountingStandardID)+"' AS AccountingType, scoa.Member AS EYGLCode, tb.UserGLCode AS UserGLCode, '"+str(CategoryID)+"' AS CategoryID, scoa.UltimateParent AS UltimateParent, tb.Unit AS Unit, 'admin' AS CreatedBy, 'admin' AS ModifiedBy, current_timestamp() AS CreatedDate, current_timestamp() AS ModifiedDate, NULL AS PresentationLayer, tb.UserGLDescription AS UserGLDescription FROM "+dbschema+".temp_tb_" + str(parameterID) + " tb JOIN "+dbschema+".temp_allperiodftpscoa_" + str(parameterID) + " scoa ON scoa.Member = tb.UserGLCode WHERE NOT EXISTS ( SELECT UserGlcode FROM "+dbschema+".scoamappingtable sm WHERE EntityID = '"+str(EntityID)+"' AND CategoryID = '"+str(CategoryID)+"' AND mapped = 1 and tb.UserGLCode=sm.UserGLCode)")

  spark.sql("INSERT INTO  "+dbschema+".temp_SCOAMappingTable_" + str(parameterID) + " SELECT DISTINCT * FROM (select distinct  EntityID,Period,accountingtype,EYGLCode,UserGLCode,CategoryID,UltimateParent,Unit BUCode,'admin' CreatedBy,'admin' modifiedby,current_timestamp() CreatedDate,current_timestamp() ModifiedDate,PresentationLayer,UserGLDescription from "+dbschema+".temp_SCOAMappingTable1_" + str(parameterID) + " where UserGLCode not in (select UserGLCode from "+dbschema+".scoamappingtable where categoryid='"+str(CategoryID)+"' and unmapped=1) UNION select entityid,reportingfrequency,accountingtype,eyglcode,userglcode,categoryid,ultimateparent,bucode,createdby,modifiedby,createddate,modifieddate,presentationlayer,usergldescription from "+dbschema+".scoamappingtable where entityid = '"+str(EntityID)+"' and  categoryid = '"+str(CategoryID)+"'and mapped=1 ) as t")
else:
  spark.sql("INSERT INTO "+dbschema+".temp_SCOAMappingTable1_" + str(parameterID) + " SELECT EntityID,ReportingFrequency,accountingtype,EYGLCode,UserGLCode,CategoryID,UltimateParent,BuCode,CreatedBy,modifiedby,CreatedDate,ModifiedDate,PresentationLayer,UserGLDescription FROM "+dbschema+".scoamappingtable where entityid = '"+str(EntityID)+"' and  categoryid = '"+str(CategoryID)+"'and mapped=1")

  spark.sql("INSERT INTO "+dbschema+".temp_SCOAMappingTable_" + str(parameterID) + " select distinct  EntityID,Period,accountingtype,EYGLCode,UserGLCode,CategoryID,UltimateParent,Unit BUCode,'admin' CreatedBy,'admin' modifiedby,current_timestamp() CreatedDate,current_timestamp() ModifiedDate,PresentationLayer,UserGLDescription from "+dbschema+".temp_SCOAMappingTable1_" + str(parameterID) + " where UserGLCode not in (select UserGLCode from "+dbschema+".scoamappingtable where categoryid='"+str(CategoryID)+"' and unmapped=1)")

  spark.sql(f""" MERGE INTO {dbschema}.temp_legacysddates_" + str(parameterID) + " AS t USING {dbschema}.temp_daterangetoperiodtype_" + str(parameterID) + " AS s ON t.currentvalue = s.tbperiodvalue AND t.periodtype = s.periodtype WHEN MATCHED THEN UPDATE SET t.periodformat = CASE WHEN s.customdaterange <> 1 OR s.customdaterange IS NULL THEN CASE RIGHT(t.legacyperiod, 2) WHEN '12' THEN 'Yearly' WHEN '06' THEN 'Halfyearly' WHEN '03' THEN 'Quarterly' WHEN '01' THEN 'Monthly' WHEN '09' THEN '9 Months' ELSE 'Monthly' END WHEN s.customdaterange = 1 THEN 'Monthly' END """) 
  

In [0]:
queryString="DROP TABLE IF EXISTS {vSQLDB}.temp_priorityseq_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_prioritylevel2_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_priorityseq1_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_prioritylevel2temp_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_tballperiod_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_tblegacymatchperiod_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_tblegacymatchperiod1_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_ytddates_" + str(parameterID) + ";"

queryString=queryString.replace("{vSQLDB}",f"{dbschema}")

sqlwarehouse_queryExecutor(queryString)

queryString="CREATE TABLE {vSQLDB}.temp_priorityseq_" + str(parameterID) + "(Period VARCHAR(20),stdperiod VARCHAR(20),PeriodType VARCHAR(50),closing VARCHAR(20),legacyperiod VARCHAR(20),numericvalueofperiod INT,priorityseq INT,Order INT,FTPPeriod INT);CREATE TABLE {vSQLDB}.temp_prioritylevel2_" + str(parameterID) + "(Period VARCHAR(20),stdperiod VARCHAR(20),PeriodType VARCHAR(50),closing VARCHAR(20),legacyperiod VARCHAR(20),numericvalueofperiod INT,priorityseq INT,Order INT);CREATE TABLE {vSQLDB}.temp_priorityseq1_" + str(parameterID) + "(Period VARCHAR(20),stdperiod VARCHAR(20),PeriodType VARCHAR(50),closing VARCHAR(20),legacyperiod VARCHAR(20),numericvalueofperiod INT,FTPPeriod INT);CREATE TABLE {vSQLDB}.temp_prioritylevel2temp_" + str(parameterID) + "(Period VARCHAR(20),stdperiod VARCHAR(20),PeriodType VARCHAR(50),closing VARCHAR(20),legacyperiod VARCHAR(20),numericvalueofperiod INT);CREATE TABLE {vSQLDB}.temp_tballperiod_" + str(parameterID) + "(Period VARCHAR(20),stdperiod VARCHAR(20),PeriodType VARCHAR(50),closing VARCHAR(20),LegacyPeriod VARCHAR(20),numericvalueofperiod INT);CREATE TABLE {vSQLDB}.temp_tblegacymatchperiod_" + str(parameterID) + "(Period VARCHAR(20),stdperiod VARCHAR(20),PeriodType VARCHAR(50),closing VARCHAR(20),LegacyPeriod VARCHAR(20),numericvalueofperiod INT);CREATE TABLE {vSQLDB}.temp_tblegacymatchperiod1_" + str(parameterID) + "(Period VARCHAR(20),stdperiod VARCHAR(20),PeriodType VARCHAR(50),closing VARCHAR(20),LegacyPeriod VARCHAR(20),numericvalueofperiod INT);CREATE TABLE {vSQLDB}.temp_ytddates_" + str(parameterID) + "(PeriodType VARCHAR(50),CurrentValue VARCHAR(6),FTPValue VARCHAR(10));"
            
queryString=queryString.replace("{vSQLDB}",f"{dbschema}")

sqlwarehouse_queryExecutor(queryString)


In [0]:
Extendedmonthdiff=0
Extendedcycleyear=""
ExtValue=""
extflag=0
countPriority=0

Extendedmonthdiff_df=spark.sql("SELECT DATEDIFF(MONTH, startdate, enddate)+ 1 FROM v_EntityPeriodDetails where fk_entitymasterid='"+str(EntityID)+"' ")
if Extendedmonthdiff_df.count()>0 :Extendedmonthdiff=Extendedmonthdiff_df.collect()[0][0]

extflag_df=spark.sql("SELECT 1 FROM v_EntityPeriodDetails where fk_entitymasterid='"+str(EntityID)+"' and isExtendedPeriod=1 ")
if extflag_df.count()>0 :extflag=extflag_df.collect()[0][0]

ExtValue=spark.sql("SELECT SUBSTRING('"+str(PeriodValue)+"',1,CHARINDEX('-','"+str(PeriodValue)+"')-1) as extValue").collect()[0][0]

Extendedcycleyear_df=spark.sql("SELECT Year FROM v_PeriodicalDatesForExtensionPeriod where Entityid='"+str(EntityID)+"' and Value="+str(extflag)+" and FinancialCycleID="+str(FiscalPeriodID)+" and Year is not null ")
if Extendedcycleyear_df.count()>0 :Extendedcycleyear=Extendedcycleyear_df.collect()[0][0]

if showlog:
    print("Extendedmonthdiff                : " , Extendedmonthdiff)
    # print("Extendedcycleyear                : " , Extendedcycleyear)
    # print("ExtValue                         : " , ExtValue)
    # print("extflag                          : " , extflag)
    # print("countPriority                    : " , countPriority)

if extflag != 1 or (extflag == 1 and Extendedmonthdiff <= 12) or (extflag == 1 and Extendedmonthdiff > 12 and Extendedcycleyear != FinancialYear):

    queryString="DROP TABLE IF EXISTS {vSQLDB}.temp_periodicaldateswithlargeperiod_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_periodicaldateswithequalperiod_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_legacyDatesLite_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_prioritylevel2temp_01_" + str(parameterID) + ";" 
    
    queryString=queryString.replace("{vSQLDB}",f"{dbschema}")
    
    sqlwarehouse_queryExecutor(queryString)


    if PeriodValueInYHQM == 0:
        spark.sql("DELETE FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " where PeriodType LIKE '%YTD'")
        spark.sql("UPDATE "+dbschema+".temp_legacysddates_" + str(parameterID) + " SET periodtype=periodtype + '_YTD' ")
    
    spark.sql("CREATE TABLE "+dbschema+".temp_periodicaldateswithlargeperiod_" + str(parameterID) + " AS SELECT DISTINCT PeriodType, closing, LegacyPeriod, numericvalueofperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN v_PeriodicalDates mp ON ld.FTPPeriod >= mp.Order WHERE financialcycleid = "+str(FiscalPeriodID)+" AND mp.closing <= ld.currentvalue AND mp.closing >= ld.openingperiodvalue")

    spark.sql("CREATE TABLE "+dbschema+".temp_periodicaldateswithequalperiod_" + str(parameterID) + " AS SELECT DISTINCT PeriodType,closing, LegacyPeriod, numericvalueofperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN v_PeriodicalDates mp ON ld.FTPPeriod = mp.Order WHERE financialcycleid = "+str(FiscalPeriodID)+" AND mp.closing <= ld.currentvalue AND mp.closing >= ld.openingperiodvalue ")

    if PeriodValueInYHQM == 0:
        spark.sql("INSERT INTO "+dbschema+".temp_tballperiod_" + str(parameterID) + " SELECT Period, LEFT(Period, 6) stdperiod, ld.*,"+alldimcols+" FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN "+dbschema+".temp_periodicaldateswithlargeperiod_" + str(parameterID) + " ld ON LEFT(tb.Period, 6)= ld.closing AND RIGHT(tb.Period, 2) <= ABS(ld.numericvalueofperiod) WHERE EntityID_fk = "+str(EntityID)+" ")

        spark.sql("INSERT INTO "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " SELECT Period, LEFT(Period, 6) stdperiod, ld.*,"+alldimcols+" FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN "+dbschema+".temp_periodicaldateswithequalperiod_" + str(parameterID) + " ld ON tb.Period = CONCAT(closing, '-', RIGHT(legacyperiod, 2)) WHERE EntityID_fk = "+str(EntityID)+" " )

        spark.sql("INSERT INTO "+dbschema+".temp_priorityseq1_" + str(parameterID) + " SELECT tb.*, ld.FTPPeriod FROM "+dbschema+".temp_tballperiod_" + str(parameterID) + " tb JOIN ( SELECT DISTINCT ld.LegacyPeriod, ld.periodtype, ld.FTPPeriod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld LEFT JOIN "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " mp ON ld.LegacyPeriod = mp.LegacyPeriod AND ld.periodtype = mp.periodtype WHERE mp.LegacyPeriod IS NULL ) ld ON LEFT(tb.Period, 6)<= LEFT(ld.LegacyPeriod, 6) AND tb.periodtype = ld.periodtype WHERE RIGHT(tb.Period, 2)<> RIGHT(ld.LegacyPeriod, 2)" )

        spark.sql("INSERT INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " SELECT Period, stdperiod, PeriodType, closing, legacyperiod, numericvalueofperiod, ROW_NUMBER() OVER (PARTITION BY stdperiod, periodtype ORDER BY Period DESC) priorityseq, CASE WHEN SUBSTRING(Period, LENGTH(Period) - 1, 2) >= '12' THEN 4 WHEN SUBSTRING(Period, LENGTH(Period) - 1, 2) = '06' THEN 3 WHEN SUBSTRING(Period, LENGTH(Period) - 1, 2) = '03' THEN 2 ELSE 1 END, FTPPeriod FROM "+dbschema+".temp_priorityseq1_" + str(parameterID) + " ORDER BY periodtype" )

        spark.sql("DELETE FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " WHERE priorityseq<>1")

        spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS t USING ( select ld.id,ps.Period from "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld inner join "+dbschema+".temp_priorityseq_" + str(parameterID) + " ps on ld.periodtype = ps.PeriodType AND ld.currentvalue = ps.stdperiod ) as s ON t.id=s.id WHEN MATCHED THEN UPDATE SET t.LegacyPeriod = s.Period")

        spark.sql("INSERT INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " SELECT PeriodType, LEFT(legacyperiod, 6) stdperiod, Period FROM "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " WHERE (Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year') UNION SELECT PeriodType, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " WHERE (Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year')")

In [0]:
if extflag != 1 or (extflag == 1 and Extendedmonthdiff <= 12) or (extflag == 1 and Extendedmonthdiff > 12 and Extendedcycleyear != FinancialYear):
    if PeriodValueInYHQM == 1:
        spark.sql("INSERT INTO "+dbschema+".temp_tballperiod_" + str(parameterID) + " SELECT Period, LEFT(Period, 6) stdperiod, ld.* FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN "+dbschema+".temp_periodicaldateswithlargeperiod_" + str(parameterID) + " ld ON LEFT(tb.Period, 6)= ld.closing AND RIGHT(tb.Period, 2) <= ABS(ld.numericvalueofperiod) WHERE EntityID_fk = "+str(EntityID)+" ")

        spark.sql("INSERT INTO "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " SELECT Period, LEFT(Period, 6) stdperiod, ld.* FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN "+dbschema+".temp_periodicaldateswithequalperiod_" + str(parameterID) + " ld ON tb.Period = CONCAT(closing, '-', RIGHT(legacyperiod, 2)) WHERE EntityID_fk = "+str(EntityID)+" " )

        spark.sql("DELETE FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period,6)=LEFT(LegacyPeriod,6) AND CAST(RIGHT(Period,2) AS INT)<>ABS(NumericValueofPeriod)")

        spark.sql("DELETE FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period,6)<>LEFT(LegacyPeriod,6) AND CAST(RIGHT(Period,2) AS INT)<ABS(NumericValueofPeriod)")

        spark.sql("INSERT INTO "+dbschema+".temp_tblegacymatchPeriod1_" + str(parameterID) + " SELECT * FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + "")

        spark.sql("DELETE FROM "+dbschema+".temp_tblegacymatchPeriod1_" + str(parameterID) + " WHERE LEFT(Period, 6) <> LEFT(LegacyPeriod, 6) AND CAST(RIGHT(Period, 2) AS INT) = ABS(NumericValueofPeriod)")

        spark.sql("INSERT INTO "+dbschema+".temp_priorityseq1_" + str(parameterID) + " SELECT tb.*, ld.FTPPeriod FROM "+dbschema+".temp_tballperiod_" + str(parameterID) + " tb JOIN (SELECT DISTINCT ls.legacyperiod, ls.periodtype, CONCAT( LEFT( DATE_ADD( MONTH, -CAST(RIGHT(ls.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ls.LegacyPeriod, 6), 2), '-01-', LEFT(ls.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 4 ),RIGHT(LEFT(DATEADD(MONTH, -CAST(RIGHT(ls.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ls.LegacyPeriod, 6), 2), '-01-', LEFT(ls.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 7), 2))  AS fromperiod, ls.FTPPeriod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ls LEFT JOIN "+dbschema+".temp_tblegacymatchPeriod1_" + str(parameterID) + " mp ON ls.LegacyPeriod = mp.LegacyPeriod AND ls.periodtype = mp.periodtype WHERE mp.LegacyPeriod IS NULL) ld ON LEFT(tb.Period, 6) <= LEFT(ld.LegacyPeriod, 6) AND LEFT(tb.Period, 6) > fromperiod AND tb.periodtype = ld.periodtype WHERE RIGHT(tb.Period, 2) <> RIGHT(ld.LegacyPeriod, 2)")

        spark.sql("INSERT INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " SELECT   Period,stdperiod,PeriodType,closing,legacyperiod,numericvalueofperiod, Row_number() OVER( partition BY stdperiod, periodtype ORDER BY Period DESC ) priorityseq, CASE WHEN RIGHT(Period, 2)>= '12' THEN 5 WHEN RIGHT(Period, 2)>= '09' THEN 4 WHEN RIGHT(Period, 2)= '06' THEN 3 WHEN RIGHT(Period, 2)= '03' THEN 2 WHEN RIGHT(Period, 2)= '01' THEN 1 ELSE 1 END, FTPPeriod FROM "+dbschema+".temp_priorityseq1_" + str(parameterID) + " ORDER BY periodtype")
        
        spark.sql("CREATE TABLE "+dbschema+".temp_prioritylevel2temp_01_" + str(parameterID) + " as SELECT periodtype, ld.Period, ld.legacyperiod, ftpperiod, concat(pd.closing, CASE ftpperiod WHEN 5 THEN + '-12' WHEN 4 THEN + '-09' WHEN 3 THEN + '-06' WHEN 2 THEN + '-03' WHEN 1 THEN + '-01' END) closing FROM v_PeriodicalDates pd JOIN (SELECT tbm.periodtype, Max(tbm.Period) Period, ld.FinancialYear, ld.LegacyPeriod, CASE ftpperiod WHEN 1 THEN ftpperiod ELSE ftpperiod - 1 END AS ftpperiod, CONCAT( LEFT( DATE_ADD( MONTH, -CAST(RIGHT(ld.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ld.LegacyPeriod, 6), 2), '-01-', LEFT(ld.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 4 ),RIGHT(LEFT(DATEADD(MONTH, -CAST(RIGHT(ld.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ld.LegacyPeriod, 6), 2), '-01-', LEFT(ld.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 7), 2)) AS fromperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " tbm ON ld.periodtype = tbm.periodtype AND ld.legacyperiod = tbm.legacyperiod GROUP BY tbm.periodtype, ld.FinancialYear, ld.LegacyPeriod,ftpperiod) ld ON pd.closing > fromperiod AND pd.closing <= LEFT(LegacyPeriod, 6) AND pd.Order = ld.FTPPeriod AND pd.Year = ld.FinancialYear WHERE financialcycleid = "+str(FiscalPeriodID)+" AND ld.legacyperiod <> ld.Period")

        spark.sql("INSERT INTO "+dbschema+".temp_prioritylevel2temp_" + str(parameterID) + " SELECT tb.Period, LEFT(tb.Period, 6) AS stdperiod, periodtype, LEFT(tb.Period, 6) AS closing, legacyperiod, ftpperiod FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN "+dbschema+".temp_prioritylevel2temp_01_" + str(parameterID) + " pt ON tb.Period = pt.closing WHERE EntityID_fk = "+str(EntityID)+" ")

        spark.sql("INSERT INTO " + dbschema + ".temp_prioritylevel2_" + str(parameterID) + " SELECT *, Row_number() OVER( partition BY stdperiod, periodtype ORDER BY Period DESC ) priorityseq, CASE WHEN RIGHT(Period, 2)>= '12' THEN 5 WHEN RIGHT(Period, 2)= '09' THEN 4 WHEN RIGHT(Period, 2)= '06' THEN 3 WHEN RIGHT(Period, 2)= '03' THEN 2 WHEN RIGHT(Period, 2)= '01' THEN 1 ELSE 1 END AS order FROM " + dbschema + ".temp_prioritylevel2temp_" + str(parameterID) + " ORDER BY periodtype")

        spark.sql("DELETE FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " WHERE priorityseq<>1")

        spark.sql("DELETE FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " WHERE priorityseq<>1")

        df_temp_001=spark.sql("SELECT DISTINCT PeriodType, max(order) order FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " GROUP BY PeriodType EXCEPT SELECT ps.periodtype, ps.order FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " ps JOIN "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.Order = ld.ftpperiod - 1")

        if df_temp_001.count()>0 :

            spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_ctePS_" + str(parameterID) + "")

            spark.sql("CREATE TABLE "+dbschema+".temp_ctePS_" + str(parameterID) + " AS SELECT PeriodType,LegacyPeriod,max(Order) Order FROM "+dbschema+".temp_Priorityseq_" + str(parameterID) + " GROUP BY PeriodType,LegacyPeriod")

            spark.sql("MERGE INTO " + dbschema + ".temp_legacysddates_" + str(parameterID) + " AS ld USING " + dbschema + ".temp_ctePS_" + str(parameterID) + " AS ps ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.order <> ld.ftpperiod WHEN MATCHED THEN UPDATE SET ld.ftpperiod = ps.order + 1 ")

        spark.sql("MERGE INTO " + dbschema + ".temp_tblegacymatchperiod_" + str(parameterID) + " AS tbm USING " + dbschema + ".temp_prioritylevel2_" + str(parameterID) + " AS pl ON tbm.periodtype = pl.periodtype AND tbm.legacyperiod = pl.legacyperiod WHEN MATCHED THEN DELETE")

        spark.sql("MERGE INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " AS ps USING "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.Order <> ld.ftpperiod - 1 WHEN MATCHED THEN DELETE")

        spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_bspriority_" + str(parameterID) + "")

        spark.sql("CREATE TABLE "+dbschema+".temp_bspriority_" + str(parameterID) + " AS SELECT PeriodType, legacyperiod, currentvalue, Max(Period) Period, Max(stdperiod) stdperiod FROM (SELECT ldd.PeriodType, ldd.legacyperiod, ldd.currentvalue, MAX(COALESCE(pseq.Period, pl2.Period, lm.Period)) AS Period, MAX(COALESCE(pseq.stdperiod, pl2.stdperiod, lm.stdperiod)) AS stdperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ldd LEFT JOIN "+dbschema+".temp_priorityseq_" + str(parameterID) + " pseq ON ldd.periodtype = pseq.PeriodType AND ldd.legacyperiod = pseq.legacyperiod LEFT JOIN "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " pl2 ON ldd.periodtype = pl2.PeriodType AND ldd.legacyperiod = pl2.legacyperiod LEFT JOIN "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " lm ON ldd.PeriodType = lm.Periodtype AND ldd.legacyperiod = lm.legacyperiod GROUP BY ldd.PeriodType, ldd.legacyperiod, ldd.currentvalue) as a WHERE a.Period is not null GROUP BY PeriodType,legacyperiod,currentvalue")

        spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld USING "+dbschema+".temp_bspriority_" + str(parameterID) + " AS ps ON ld.periodtype = ps.PeriodType AND ld.legacyperiod = ps.legacyperiod WHEN MATCHED THEN UPDATE SET ld.LegacyPeriod = ps.Period, ld.CurrentValue = ps.stdperiod")

        spark.sql("MERGE INTO "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " AS tb  USING ( SELECT * FROM ( SELECT * FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period, 6) <> LEFT(LegacyPeriod, 6)  AND CAST(RIGHT(Period, 2) AS INT) = ABS(NumericValueofPeriod) ) a  WHERE NOT EXISTS ( SELECT LegacyPeriod  FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " b  WHERE a.LegacyPeriod = Period ) ) del  ON tb.LegacyPeriod = del.LegacyPeriod  WHEN MATCHED THEN DELETE" )

        if countofcomp == 0 or countofcomp  == 1 :
            spark.sql("INSERT INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " WHERE ( Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year' ) UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " WHERE ( Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year' ) UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " WHERE ( Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year' )")
        
        if countofcomp > 1 :
            spark.sql("INSERT INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " SELECT periodtype, pcurrentvalue, legacyperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE periodtype NOT IN ('currentPeriod', 'PreviousPeriod') EXCEPT SELECT periodtype, pcurrentvalue, legacyperiod FROM (  SELECT periodtype, pcurrentvalue, legacyperiod  FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + "  WHERE left(legacyperiod, 6) <> pcurrentvalue ) a WHERE NOT EXISTS (  SELECT legacyPeriod  FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " b  WHERE a.legacyPeriod = Period  AND a.periodtype = PeriodType ) UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " ")

In [0]:
if extflag==1 and Extendedmonthdiff<=12 :

    spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld USING v_PeriodicalDatesForExtensionPeriod AS pd ON ld.currentvalue = pd.closing AND pd.Value = ld.Value AND financialyear = '"+str(Extendedcycleyear)+"' AND entityid = "+str(EntityID)+" WHEN MATCHED THEN UPDATE SET  p = '', ppp = '', py = '', pypp = '', ppy = '', p3y = '', p4y = '' ")

    spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS t USING ( SELECT MIN(currentvalue) AS min_currentvalue FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE p = '' ) AS s ON t.currentvalue < s.min_currentvalue WHEN MATCHED THEN DELETE")

if extflag==1 and Extendedmonthdiff > 12 :
    extopen=""
    extclose=""
    monthdiff=0
    NumericalValueofExtPeriodType=0
    exttype=""
    ExtValue=spark.sql("SELECT SUBSTRING('"+str(PeriodValue)+"',1,CHARINDEX('-','"+str(PeriodValue)+"')-1) as ExtValue").collect()[0][0]

    spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld  USING v_PeriodicalDatesForExtensionPeriod pd  ON pd.closing = ld.Pcurrentvalue AND pd.Value = ld.Value AND financialcycleid = "+str(FiscalPeriodID)+" AND entityid = "+str(EntityID)+" AND financialyear = '"+str(Extendedcycleyear)+"' WHEN MATCHED THEN  UPDATE SET Value = pd.Value, financialyear =  pd.Year, openingperiodvalue = pd.opening, CurrentValue = PcurrentValue ")

    ext_df=spark.sql("SELECT opening,closing,Year FROM v_PeriodicalDatesForExtensionPeriod WHERE entityid = "+str(EntityID)+" and Value = '"+str(ExtValue)+"' and financialcycleid="+str(FiscalPeriodID)+" ")

    if ext_df.count()>0 : 
        extopen=ext_df.collect()[0][0]
        extclose=ext_df.collect()[0][0]
        Extendedcycleyear=ext_df.collect()[0][0]

    spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld USING v_PeriodicalDatesForExtensionPeriod AS pd ON ld.currentvalue = pd.closing AND pd.Value = ld.Value and entityid = "+str(EntityID)+" and  financialyear='"+str(Extendedcycleyear)+"' WHEN MATCHED THEN UPDATE SET openingperiodvalue = '"+str(extopen)+"' , CastingPeriod = concat(ld.Value,'-','"+str(extopen)+"','-',currentvalue),legacyperiod = CASE WHEN ld.Value IN ('Q1', 'H1', 'Year', '9M') THEN CASE WHEN LENGTH(DATE_DIFF(MONTH,to_date(CONCAT('01-', RIGHT(opening, 2), '-', LEFT(opening, 4)), 'dd-MM-yyyy'),to_date(CONCAT('01-', RIGHT(closing, 2), '-', LEFT(closing, 4)), 'dd-MM-yyyy'))+1) = 1 THEN CONCAT(pcurrentvalue , '-0',DATE_DIFF(MONTH,to_date(concat('01-',RIGHT(opening, 2),'-',LEFT(opening, 4)),'dd-MM-yyyy'),to_date(concat('01-',RIGHT(closing, 2),'-',LEFT(closing, 4)),'dd-MM-yyyy'))+1)  ELSE CONCAT(pcurrentvalue , '-',DATE_DIFF(MONTH,to_date(concat('01-',RIGHT(opening, 2),'-',LEFT(opening, 4)),'dd-MM-yyyy'),to_date(concat('01-',RIGHT(closing, 2),'-',LEFT(closing, 4)),'dd-MM-yyyy'))+1) END WHEN ld.Value IN ('H2') THEN concat(currentvalue , '-' ,'06') WHEN ld.Value IN ('Q2', 'Q3', 'Q4') THEN concat(currentvalue, '-' ,'03') ELSE legacyperiod END , p = '', ppp = '', py = '', pypp = '', ppy = '', p3y = '', p4y = '' ")

    spark.sql("DELETE FROM " + dbschema + ".temp_legacysddates_" + str(parameterID) + " WHERE currentvalue < (SELECT MIN(currentvalue) FROM " + dbschema + ".temp_legacysddates_" + str(parameterID) + " WHERE p = '')")

    spark.sql("UPDATE "+dbschema+".temp_legacysddates_" + str(parameterID) + " SET numericvalueofperiod = -CAST(RIGHT(LegacyPeriod, 2) AS INT)")

    ################## Handle YTD calc for ext financial cycle ##################

    queryString="DROP TABLE IF EXISTS {vSQLDB}.temp_legacyMatchPeriodFOrLargerFTP;DROP TABLE IF EXISTS {vSQLDB}.temp_legacyMatchPeriodFOrEqualFTP;DROP TABLE IF EXISTS {vSQLDB}.temp_numperiod;DROP TABLE IF EXISTS {vSQLDB}.temp_cteorder;" 
    
    queryString=queryString.replace("{vSQLDB}",f"{dbschema}")
    
    sqlwarehouse_queryExecutor(queryString)

    spark.sql("CREATE TABLE "+dbschema+".temp_numperiod_" + str(parameterID) + " ( PeriodType VARCHAR(50), LegacyPeriod VARCHAR(20), numericvalueofperiod INT )")

    if Extendedcycleyear == FinancialYear :

        queryString="TRUNCATE TABLE {vSQLDB}.temp_priorityseq_" + str(parameterID) + ";TRUNCATE TABLE {vSQLDB}.temp_prioritylevel2_" + str(parameterID) + ";TRUNCATE TABLE {vSQLDB}.temp_priorityseq1_" + str(parameterID) + ";TRUNCATE TABLE {vSQLDB}.temp_tballperiod_" + str(parameterID) + ";TRUNCATE TABLE {vSQLDB}.temp_tblegacymatchperiod_" + str(parameterID) + ";TRUNCATE TABLE  {vSQLDB}.temp_tblegacymatchPeriod1_" + str(parameterID) + ";"
        
        queryString=queryString.replace("{vSQLDB}",f"{dbschema}")
        
        sqlwarehouse_queryExecutor(queryString)

        spark.sql("CREATE TABLE  "+dbschema+".temp_legacyMatchPeriodFOrLargerFTP_" + str(parameterID) + " AS SELECT DISTINCT PeriodType, closing, LegacyPeriod, numericvalueofperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN v_PeriodicalDatesForExtensionPeriod mp ON mp.closing <= ld.currentvalue AND mp.opening >= ld.openingperiodvalue AND ld.FTPPeriod >= mp.Order WHERE entityid = "+str(EntityID)+"  and financialcycleid="+str(FiscalPeriodID)+" ")

        spark.sql("CREATE TABLE "+dbschema+".temp_legacyMatchPeriodFOrEqualFTP_" + str(parameterID) + " AS SELECT DISTINCT PeriodType, closing, LegacyPeriod, numericvalueofperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN v_PeriodicalDatesForExtensionPeriod mp ON mp.closing <= ld.currentvalue AND mp.opening >= ld.openingperiodvalue AND ld.FTPPeriod = mp.Order WHERE entityid = "+str(EntityID)+"  and financialcycleid="+str(FiscalPeriodID)+" ")

        spark.sql("INSERT INTO "+dbschema+".temp_tballperiod_" + str(parameterID) + " SELECT tb.Period, LEFT(tb.Period, 6) AS stdperiod,ld.*,"+alldimcols+"  FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN "+dbschema+".temp_legacyMatchPeriodFOrLargerFTP_" + str(parameterID) + " ld ON LEFT(tb.Period, 6)= ld.closing WHERE EntityID_fk = "+str(EntityID)+" ")

        spark.sql("INSERT INTO "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " SELECT tb.Period, LEFT(tb.Period, 6) AS stdperiod,ld.*,"+alldimcols+"  FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN "+dbschema+".temp_legacyMatchPeriodFOrEqualFTP_" + str(parameterID) + " ld ON LEFT(tb.Period, 6)= ld.closing WHERE EntityID_fk = "+str(EntityID)+" ")

        spark.sql("INSERT INTO "+dbschema+".temp_numperiod_" + str(parameterID) + " SELECT periodtype, LegacyPeriod, MAX(numericvalueofperiod) AS numericvalueofperiod FROM ( SELECT ld.periodtype, md.LegacyPeriod, CASE LEFT(Value, 1) WHEN 'Y' THEN INT("+str(Extendedmonthdiff)+") WHEN '9' THEN INT("+str(Extendedmonthdiff)+") - 3 WHEN 'H' THEN INT("+str(Extendedmonthdiff)+") - 6 WHEN 'Q' THEN INT("+str(Extendedmonthdiff)+") - 9 ELSE 1 END AS numericvalueofperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " md ON ld.periodtype = md.PeriodType AND ld.legacyperiod = md.LegacyPeriod ) a GROUP BY periodtype, LegacyPeriod")

        spark.sql("DELETE FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period,6) = LEFT(LegacyPeriod,6) AND CAST(RIGHT(Period,2) AS INT) <> ABS(NumericValueofPeriod)")

        spark.sql("DELETE FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period,6)<>LEFT(LegacyPeriod,6) AND CAST(RIGHT(Period,2) AS INT) < ABS(NumericValueofPeriod)")

        spark.sql("MERGE INTO "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " tb1 USING "+dbschema+".temp_numperiod_" + str(parameterID) + " nm ON tb1.PeriodType = nm.periodtype AND tb1.LegacyPeriod = nm.LegacyPeriod and LEFT(tb1.Period, 6) <> LEFT(tb1.LegacyPeriod, 6) AND CAST(RIGHT(tb1.Period, 2) AS INT) > ABS(nm.numericvalueofperiod) WHEN MATCHED THEN DELETE")

        spark.sql("INSERT INTO "+dbschema+".temp_priorityseq1_" + str(parameterID) + " SELECT tb.*,ld.ftpperiod FROM "+dbschema+".temp_tballperiod_" + str(parameterID) + " tb JOIN ( SELECT DISTINCT ls.legacyperiod, ls.periodtype, CONCAT( LEFT( DATE_ADD( MONTH, -CAST(RIGHT(ls.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ls.LegacyPeriod, 6), 2), '-01-', LEFT(ls.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 4 ),RIGHT(LEFT(DATEADD(MONTH, -CAST(RIGHT(ls.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ls.LegacyPeriod, 6), 2), '-01-', LEFT(ls.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 7), 2)) AS fromperiod ,ls.FTPPeriod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ls LEFT JOIN "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " mp ON ls.LegacyPeriod = mp.LegacyPeriod AND ls.periodtype = mp.periodtype WHERE mp.LegacyPeriod IS NULL  ) ld ON LEFT(tb.Period, 6)<= LEFT(ld.LegacyPeriod, 6) AND LEFT(tb.Period, 6)> fromperiod AND tb.periodtype = ld.periodtype WHERE RIGHT(tb.Period, 2)<> RIGHT(ld.LegacyPeriod, 2) and CAST(RIGHT(tb.Period, 2) AS INT)<=ABS(NumericValueofPeriod)")

        spark.sql("INSERT INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " SELECT Period,stdperiod,PeriodType,closing,legacyperiod,numericvalueofperiod, Row_number() OVER( partition BY stdperiod, periodtype ORDER BY Period DESC ) priorityseq, 0, ftpperiod FROM "+dbschema+".temp_priorityseq1_" + str(parameterID) + " ORDER BY periodtype")

In [0]:
if extflag==1 and Extendedmonthdiff > 12 :
    if Extendedcycleyear == FinancialYear :
        queryString="DROP TABLE IF EXISTS {vSQLDB}.temp_cteorder_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_prioritylevel2temp_01_" + str(parameterID) + ";" 
        
        queryString=queryString.replace("{vSQLDB}",f"{dbschema}")
        
        sqlwarehouse_queryExecutor(queryString)

        spark.sql("CREATE TABLE  "+dbschema+".temp_cteorder_" + str(parameterID) + " AS SELECT ps.Period, ps.PeriodType, mp.closing, ps.legacyperiod, priorityseq, max(mp.order) AS Order FROM (SELECT *, right(left(to_date(concat('01-',substring ( Period,charindex(' ',Period)+1, case when charindex('-',Period)=0 then len(Period)-charindex(' ',Period) else charindex('-',Period)-charindex(' ',Period)-1 end ),'-',left(closing,4)), 'dd-MM-yyyy'),7),2) FromPeriod FROM v_periodicaldatesforextensionperiod WHERE entityid="+str(EntityID)+" ) mp JOIN (SELECT *, CONCAT( LEFT( DATE_ADD( MONTH, -CAST(RIGHT(Period, 2)AS INT)+1, to_date( CONCAT( RIGHT(LEFT(Period, 6), 2), '-01-', LEFT(Period, 4) ), 'MM-dd-yyyy' ) ), 4 ),RIGHT(LEFT(DATEADD(MONTH, -CAST(RIGHT(Period, 2) AS INT)+1, to_date( CONCAT( RIGHT(LEFT(Period, 6), 2), '-01-', LEFT(Period, 4) ), 'MM-dd-yyyy' ) ), 7), 2))  AS fromperiod FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " ) ps ON ps.closing = mp.closing AND ps.FTPPeriod > mp.order AND ps.fromperiod = left(ps.FromPeriod,4) + mp.FromPeriod WHERE financialcycleid ="+str(FiscalPeriodID)+" AND entityid = "+str(EntityID)+" GROUP BY ps.Period, ps.PeriodType, mp.closing, ps.legacyperiod, priorityseq")
        
        spark.sql("MERGE INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " AS ps USING "+dbschema+".temp_cteorder_" + str(parameterID) + " AS cte ON ps.Period = cte.Period WHEN MATCHED THEN UPDATE SET ps.order = cte.order")
        
        spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_prioritylevel2temp_01_" + str(parameterID) + "")
        
        spark.sql("CREATE TABLE "+dbschema+".temp_prioritylevel2temp_01_" + str(parameterID) + " AS SELECT DISTINCT periodtype, ld.Period, ld.legacyperiod, ftpperiod, ld.closing, CASE WHEN ld.Value IN ('H1', 'Q1', 'Year', '9M') THEN Datediff(month, TO_DATE(CONCAT('01', '-', RIGHT(FromOpeningPeriod, 2), '-', LEFT(FromOpeningPeriod, 4)), 'dd-MM-yyyy'), TO_DATE(CONCAT('01', '-', RIGHT(closing, 2), '-', LEFT(closing, 4)), 'dd-MM-yyyy')) + 1 WHEN ld.Value NOT IN ('H1', 'Q1', 'Year', '9M') THEN CASE WHEN LEFT(ld.Value, 1)='9' THEN '09' WHEN LEFT(ld.Value, 1)='H' THEN '06' WHEN LEFT(ld.Value, 1)='Q' THEN '03' WHEN LEFT(ld.Value, 1)='M' THEN '01' END END as monthdiff FROM ( SELECT ld.PeriodType, ld.Period, ld.FinancialYear, ld.LegacyPeriod, ld.ftpperiod, pd.Value, pd.closing, CONCAT(left(FromPeriod, 4),right(left(to_date(concat('01-',substring ( pd.Period,charindex(' ',pd.Period)+1, case when charindex('-',pd.Period)=0 then len(pd.Period)-charindex(' ',pd.Period) else charindex('-',pd.Period)-charindex(' ',pd.Period)-1 end ),'-',left(closing,4)), 'dd-MM-yyyy'),7),2)) as FromOpeningPeriod FROM v_PeriodicalDatesForExtensionPeriod as pd JOIN ( SELECT tbm.periodtype, MAX(tbm.Period) as Period, ld.FinancialYear, ld.LegacyPeriod, CASE WHEN ftpperiod = 1 THEN ftpperiod ELSE ftpperiod - 1 END as ftpperiod, CONCAT( LEFT( DATE_ADD( MONTH, -CAST(RIGHT(ld.LegacyPeriod, 2)AS INT), to_date( CONCAT( RIGHT(LEFT(ld.LegacyPeriod, 6), 2), '-01-', LEFT(ld.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 4 ),RIGHT(LEFT(DATEADD(MONTH, -CAST(RIGHT(ld.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ld.LegacyPeriod, 6), 2), '-01-', LEFT(ld.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 7), 2))  AS fromperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " as ld JOIN "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " as tbm ON ld.periodtype = tbm.periodtype AND ld.legacyperiod = tbm.legacyperiod GROUP BY tbm.periodtype, ld.FinancialYear, ld.LegacyPeriod, ftpperiod ) as ld ON pd.closing > fromperiod AND pd.closing <= LEFT(LegacyPeriod, 6) AND pd.Order = ld.ftpperiod AND pd.Year = ld.FinancialYear WHERE financialcycleid = "+str(FiscalPeriodID)+" and Entityid = "+str(EntityID)+"  ) as ld ")

        spark.sql("INSERT INTO "+dbschema+".temp_prioritylevel2temp_" + str(parameterID) + " SELECT tb.Period, LEFT(tb.Period, 6) AS stdperiod, periodtype, LEFT(tb.Period, 6) AS closing, legacyperiod, ftpperiod,"+alldimcols+" FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN "+dbschema+".temp_prioritylevel2temp_01_" + str(parameterID) + " pt ON LEFT(tb.Period,6) = pt.closing  and ABS(RIGHT(tb.Period, 2) )= monthdiff WHERE EntityID_fk = "+str(EntityID)+" ")
        
        spark.sql("INSERT INTO " + dbschema + ".temp_prioritylevel2_" + str(parameterID) + " SELECT *, Row_number() OVER( partition BY stdperiod, periodtype ORDER BY Period DESC ) priorityseq, 0 as order FROM " + dbschema + ".temp_prioritylevel2temp_" + str(parameterID) + " ORDER BY periodtype")

        spark.sql("DELETE FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " WHERE priorityseq<>1")

        spark.sql("DELETE FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " WHERE priorityseq<>1")

        df_temp_002=spark.sql("SELECT DISTINCT PeriodType, max(order) order FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " GROUP BY PeriodType EXCEPT SELECT ps.periodtype, ps.order FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " ps JOIN "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.Order = ld.ftpperiod - 1")

        if df_temp_002.count()>0 :

            spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_ctePS_" + str(parameterID) + "")

            spark.sql("CREATE TABLE "+dbschema+".temp_ctePS_" + str(parameterID) + " AS SELECT PeriodType,LegacyPeriod,max(Order) Order FROM "+dbschema+".temp_Priorityseq_" + str(parameterID) + " GROUP BY PeriodType,LegacyPeriod")
            
            spark.sql("MERGE INTO " + dbschema + ".temp_legacysddates_" + str(parameterID) + " AS ld USING " + dbschema + ".temp_ctePS_" + str(parameterID) + " AS ps ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.order <> ld.ftpperiod WHEN MATCHED THEN UPDATE SET ld.ftpperiod = ps.order + 1 ")

        spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_deleteplev2_" + str(parameterID) + "")
        
        spark.sql("CREATE TABLE  "+dbschema+".temp_deleteplev2_" + str(parameterID) + " AS SELECT LegacyPeriod,PeriodType from "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " where Period=LegacyPeriod")

        spark.sql("MERGE INTO "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " AS pl USING "+dbschema+".temp_deleteplev2_" + str(parameterID) + " AS tbm ON tbm.periodtype = pl.periodtype AND tbm.legacyperiod = pl.legacyperiod WHEN MATCHED THEN DELETE")
        
        spark.sql("MERGE INTO "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " AS tbm USING "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " AS pl ON tbm.periodtype = pl.periodtype AND tbm.legacyperiod = pl.legacyperiod WHEN MATCHED THEN DELETE")
        
        spark.sql("MERGE INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " AS ps USING "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.order <> ld.ftpperiod - 1 WHEN MATCHED THEN DELETE")

        spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_bspriority_" + str(parameterID) + "")
        
        spark.sql("CREATE TABLE "+dbschema+".temp_bspriority_" + str(parameterID) + " AS SELECT PeriodType, legacyperiod, currentvalue, MAX(Period) AS Period, MAX(stdperiod) AS stdperiod FROM ( SELECT ld.PeriodType, ld.legacyperiod, ld.currentvalue, MAX(ps.Period) AS Period, MAX(ps.stdperiod) AS stdperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN "+dbschema+".temp_priorityseq_" + str(parameterID) + " ps ON ld.periodtype = ps.PeriodType AND ld.legacyperiod = ps.legacyperiod GROUP BY ld.PeriodType, ld.legacyperiod, ld.currentvalue UNION SELECT ld.PeriodType, ld.legacyperiod, ld.currentvalue, MAX(ps.Period) AS Period, MAX(ps.stdperiod) AS stdperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " ps ON ld.periodtype = ps.PeriodType AND ld.legacyperiod = ps.legacyperiod GROUP BY ld.PeriodType, ld.legacyperiod, ld.currentvalue UNION SELECT PeriodType, legacyperiod, LEFT(legacyperiod, 6), MAX(Period), MAX(stdperiod) FROM "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " GROUP BY PeriodType, legacyperiod, legacyperiod ) a GROUP BY PeriodType, legacyperiod, currentvalue ")

        spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld USING "+dbschema+".temp_bspriority_" + str(parameterID) + " AS ps ON ld.periodtype = ps.PeriodType AND ld.legacyperiod = ps.legacyperiod WHEN MATCHED THEN UPDATE SET ld.LegacyPeriod = ps.Period, ld.CurrentValue = ps.stdperiod")

        spark.sql("MERGE INTO "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " AS tb  USING ( SELECT * FROM ( SELECT * FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period, 6) <> LEFT(LegacyPeriod, 6) ) a  WHERE NOT EXISTS ( SELECT LegacyPeriod  FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " b  WHERE a.LegacyPeriod = Period ) ) del  ON tb.LegacyPeriod = del.LegacyPeriod  WHEN MATCHED THEN DELETE" )

        spark.sql("TRUNCATE TABLE "+dbschema+".temp_ytddates_" + str(parameterID) + "")

        if countofcomp == 0 or countofcomp  == 1 :
            spark.sql("INSERT INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " WHERE ( Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year' ) UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " WHERE ( Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year' ) UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " WHERE ( Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year' )")
        
        if countofcomp > 1 :
            spark.sql("INSERT INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " SELECT periodtype, pcurrentvalue, legacyperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE periodtype NOT IN ('currentPeriod', 'PreviousPeriod') EXCEPT SELECT periodtype, pcurrentvalue, legacyperiod FROM (  SELECT periodtype, pcurrentvalue, legacyperiod  FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + "  WHERE left(legacyperiod, 6) <> pcurrentvalue ) a WHERE NOT EXISTS (  SELECT legacyPeriod  FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " b  WHERE a.legacyPeriod = Period  AND a.periodtype = PeriodType ) UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " UNION select periodtype,pcurrentvalue,period from ( select periodtype, LEFT(legacyperiod, 6) as pcurrentvalue,period  from  "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " except select  periodtype, pcurrentvalue,legacyperiod  from "+dbschema+".temp_legacysddates_" + str(parameterID) + ") b ")



In [0]:
if extflag==1 and Extendedmonthdiff > 12 :
    if Extendedcycleyear != FinancialYear :
        
        queryString="TRUNCATE TABLE {vSQLDB}.temp_priorityseq1_" + str(parameterID) + ";TRUNCATE TABLE {vSQLDB}.temp_tballperiod_" + str(parameterID) + ";TRUNCATE TABLE {vSQLDB}.temp_tblegacymatchperiod_" + str(parameterID) + ";TRUNCATE TABLE {vSQLDB}.temp_prioritylevel2temp_" + str(parameterID) + ";"
        
        queryString=queryString.replace("{vSQLDB}",f"{dbschema}")
        
        sqlwarehouse_queryExecutor(queryString)
        
        spark.sql("MERGE INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " AS ps USING "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld ON ps.periodtype = ld.periodtype AND financialyear='"+str(Extendedcycleyear)+"'  WHEN MATCHED THEN DELETE")

        spark.sql("MERGE INTO "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " AS ps USING "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld ON ps.periodtype = ld.periodtype AND financialyear='"+str(Extendedcycleyear)+"'  WHEN MATCHED THEN DELETE")

        spark.sql("SELECT DISTINCT Period, LEFT(Period, 6) AS stdperiod, ld.* FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN ( SELECT DISTINCT PeriodType, closing, LegacyPeriod, numericvalueofperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN v_PeriodicalDatesForExtensionPeriod mp ON mp.closing <= ld.currentvalue AND mp.opening >= ld.openingperiodvalue AND ld.FTPPeriod >= mp.Order WHERE financialcycleid = "+str(FiscalPeriodID)+" AND Entityid="+str(EntityID)+") ld ON LEFT(tb.Period, 6) = ld.closing  WHERE EntityID_fk = "+str(EntityID)+" ")

        spark.sql("INSERT INTO "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " SELECT Period, LEFT(Period, 6) AS stdperiod, ld.*,"+alldimcols+" FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " tb JOIN ( SELECT DISTINCT PeriodType, closing, LegacyPeriod, numericvalueofperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN v_PeriodicalDatesForExtensionPeriod mp ON mp.closing <= ld.currentvalue AND mp.opening >= ld.openingperiodvalue AND ld.FTPPeriod = mp.Order WHERE financialcycleid = "+str(FiscalPeriodID)+"  AND Entityid = "+str(EntityID)+" ) ld ON LEFT(tb.Period, 6) = closing WHERE EntityID_fk = "+str(EntityID)+" ORDER BY PeriodType")

        spark.sql("INSERT INTO "+dbschema+".temp_numperiod_" + str(parameterID) + " SELECT periodtype, LegacyPeriod, MAX(numericvalueofperiod) AS numericvalueofperiod FROM ( SELECT ld.periodtype, md.LegacyPeriod, CASE LEFT(Value, 1) WHEN 'Y' THEN INT("+str(Extendedmonthdiff)+") WHEN '9' THEN INT("+str(Extendedmonthdiff)+") - 3 WHEN 'H' THEN INT("+str(Extendedmonthdiff)+") - 6 WHEN 'Q' THEN INT("+str(Extendedmonthdiff)+") - 9 ELSE 1 END AS numericvalueofperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " md ON ld.periodtype = md.PeriodType AND ld.legacyperiod = md.LegacyPeriod ) a GROUP BY periodtype, LegacyPeriod")

        spark.sql("DELETE FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period,6) = LEFT(LegacyPeriod,6) AND CAST(RIGHT(Period,2) AS INT) <> ABS(NumericValueofPeriod)")
        
        spark.sql("DELETE FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period,6)<>LEFT(LegacyPeriod,6) AND CAST(RIGHT(Period,2) AS INT) < ABS(NumericValueofPeriod)")
        
        spark.sql("MERGE INTO "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " tb1 USING "+dbschema+".temp_numperiod_" + str(parameterID) + " nm ON tb1.PeriodType = nm.periodtype AND tb1.LegacyPeriod = nm.LegacyPeriod and LEFT(tb1.Period, 6) <> LEFT(tb1.LegacyPeriod, 6) AND CAST(RIGHT(tb1.Period, 2) AS INT) > ABS(nm.numericvalueofperiod) WHEN MATCHED THEN DELETE")

        spark.sql("INSERT INTO "+dbschema+".temp_priorityseq1_" + str(parameterID) + " SELECT tb.*,ld.ftpperiod FROM "+dbschema+".temp_tballperiod_" + str(parameterID) + " tb JOIN ( SELECT DISTINCT ls.legacyperiod, ls.periodtype, CONCAT( LEFT( DATE_ADD( MONTH, -CAST(RIGHT(ls.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ls.LegacyPeriod, 6), 2), '-01-', LEFT(ls.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 4 ),RIGHT(LEFT(DATEADD(MONTH, -CAST(RIGHT(ls.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ls.LegacyPeriod, 6), 2), '-01-', LEFT(ls.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 7), 2)) AS fromperiod ,ls.FTPPeriod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ls LEFT JOIN "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " mp ON ls.LegacyPeriod = mp.LegacyPeriod AND ls.periodtype = mp.periodtype WHERE mp.LegacyPeriod IS NULL  ) ld ON LEFT(tb.Period, 6)<= LEFT(ld.LegacyPeriod, 6) AND LEFT(tb.Period, 6)> fromperiod AND tb.periodtype = ld.periodtype WHERE RIGHT(tb.Period, 2)<> RIGHT(ld.LegacyPeriod, 2) and CAST(RIGHT(tb.Period, 2) AS INT)<=ABS(NumericValueofPeriod)")

        spark.sql("INSERT INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " SELECT Period,stdperiod,PeriodType,closing,legacyperiod,numericvalueofperiod, Row_number() OVER( partition BY stdperiod, periodtype ORDER BY Period DESC ) priorityseq, 0, ftpperiod FROM "+dbschema+".temp_priorityseq1_" + str(parameterID) + " ORDER BY periodtype")

        queryString="DROP TABLE IF EXISTS {vSQLDB}.temp_cteorder_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_prioritylevel2temp_01_" + str(parameterID) + ";" 
        
        queryString=queryString.replace("{vSQLDB}",f"{dbschema}")
        
        sqlwarehouse_queryExecutor(queryString)

        spark.sql("CREATE TABLE  "+dbschema+".temp_cteorder_" + str(parameterID) + " AS SELECT ps.Period, ps.PeriodType, mp.closing, ps.legacyperiod, priorityseq, max(mp.order) AS Order FROM (SELECT *, right(left(to_date(concat('01-',substring ( Period,charindex(' ',Period)+1, case when charindex('-',Period)=0 then len(Period)-charindex(' ',Period) else charindex('-',Period)-charindex(' ',Period)-1 end ),'-',left(closing,4)), 'dd-MM-yyyy'),7),2) FromPeriod FROM v_periodicaldatesforextensionperiod WHERE entityid="+str(EntityID)+" ) mp JOIN (SELECT *, CONCAT( LEFT( DATE_ADD( MONTH, -CAST(RIGHT(Period, 2)AS INT)+1, to_date( CONCAT( RIGHT(LEFT(Period, 6), 2), '-01-', LEFT(Period, 4) ), 'MM-dd-yyyy' ) ), 4 ),RIGHT(LEFT(DATEADD(MONTH, -CAST(RIGHT(Period, 2) AS INT)+1, to_date( CONCAT( RIGHT(LEFT(Period, 6), 2), '-01-', LEFT(Period, 4) ), 'MM-dd-yyyy' ) ), 7), 2))  AS fromperiod FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " ) ps ON ps.closing = mp.closing AND ps.FTPPeriod > mp.order AND ps.fromperiod = left(ps.FromPeriod,4) + mp.FromPeriod WHERE financialcycleid ="+str(FiscalPeriodID)+" AND entityid = "+str(EntityID)+" GROUP BY ps.Period, ps.PeriodType, mp.closing, ps.legacyperiod, priorityseq")

        spark.sql("MERGE INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " AS ps USING "+dbschema+".temp_cteorder_" + str(parameterID) + " AS cte ON ps.Period = cte.Period WHEN MATCHED THEN UPDATE SET ps.order = cte.order")

        spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_prioritylevel2temp_01_" + str(parameterID) + "")

        spark.sql("CREATE TABLE "+dbschema+".temp_prioritylevel2temp_01_" + str(parameterID) + " AS SELECT DISTINCT periodtype, ld.Period, ld.legacyperiod, ftpperiod, ld.closing, CASE WHEN ld.Value IN ('H1', 'Q1', 'Year', '9M') THEN Datediff(month, TO_DATE(CONCAT('01', '-', RIGHT(FromOpeningPeriod, 2), '-', LEFT(FromOpeningPeriod, 4)), 'dd-MM-yyyy'), TO_DATE(CONCAT('01', '-', RIGHT(closing, 2), '-', LEFT(closing, 4)), 'dd-MM-yyyy')) + 1 WHEN ld.Value NOT IN ('H1', 'Q1', 'Year', '9M') THEN CASE WHEN LEFT(ld.Value, 1)='9' THEN '09' WHEN LEFT(ld.Value, 1)='H' THEN '06' WHEN LEFT(ld.Value, 1)='Q' THEN '03' WHEN LEFT(ld.Value, 1)='M' THEN '01' END END as monthdiff FROM ( SELECT ld.PeriodType, ld.Period, ld.FinancialYear, ld.LegacyPeriod, ld.ftpperiod, pd.Value, pd.closing, CONCAT(left(FromPeriod, 4),right(left(to_date(concat('01-',substring ( pd.Period,charindex(' ',pd.Period)+1, case when charindex('-',pd.Period)=0 then len(pd.Period)-charindex(' ',pd.Period) else charindex('-',pd.Period)-charindex(' ',pd.Period)-1 end ),'-',left(closing,4)), 'dd-MM-yyyy'),7),2)) as FromOpeningPeriod FROM v_PeriodicalDatesForExtensionPeriod as pd JOIN ( SELECT tbm.periodtype, MAX(tbm.Period) as Period, ld.FinancialYear, ld.LegacyPeriod, CASE WHEN ftpperiod = 1 THEN ftpperiod ELSE ftpperiod - 1 END as ftpperiod, CONCAT( LEFT( DATE_ADD( MONTH, -CAST(RIGHT(ld.LegacyPeriod, 2)AS INT), to_date( CONCAT( RIGHT(LEFT(ld.LegacyPeriod, 6), 2), '-01-', LEFT(ld.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 4 ),RIGHT(LEFT(DATEADD(MONTH, -CAST(RIGHT(ld.LegacyPeriod, 2) AS INT), to_date( CONCAT( RIGHT(LEFT(ld.LegacyPeriod, 6), 2), '-01-', LEFT(ld.LegacyPeriod, 4) ), 'MM-dd-yyyy' ) ), 7), 2))  AS fromperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " as ld JOIN "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " as tbm ON ld.periodtype = tbm.periodtype AND ld.legacyperiod = tbm.legacyperiod GROUP BY tbm.periodtype, ld.FinancialYear, ld.LegacyPeriod, ftpperiod ) as ld ON pd.closing > fromperiod AND pd.closing <= LEFT(LegacyPeriod, 6) AND pd.Order = ld.ftpperiod AND pd.Year = ld.FinancialYear WHERE financialcycleid = "+str(FiscalPeriodID)+" and Entityid = "+str(EntityID)+"  ) as ld ")

        spark.sql("INSERT INTO " + dbschema + ".temp_prioritylevel2_" + str(parameterID) + " SELECT *, Row_number() OVER( partition BY stdperiod, periodtype ORDER BY Period DESC ) priorityseq, 0 as order FROM " + dbschema + ".temp_prioritylevel2temp_" + str(parameterID) + " ORDER BY periodtype")

        spark.sql("DELETE FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " WHERE priorityseq<>1")

        spark.sql("DELETE FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " WHERE priorityseq<>1")

        df_temp_003=spark.sql("SELECT DISTINCT PeriodType, max(order) order FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " GROUP BY PeriodType EXCEPT SELECT ps.periodtype, ps.order FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " ps JOIN "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.Order = ld.ftpperiod - 1")

        if df_temp_003.count()>0 :
            spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_ctePS_" + str(parameterID) + "")
            
            spark.sql("CREATE TABLE "+dbschema+".temp_ctePS_" + str(parameterID) + " AS SELECT PeriodType,LegacyPeriod,max(Order) Order FROM "+dbschema+".temp_Priorityseq_" + str(parameterID) + " GROUP BY PeriodType,LegacyPeriod")
            
            spark.sql("MERGE INTO " + dbschema + ".temp_legacysddates_" + str(parameterID) + " AS ld USING " + dbschema + ".temp_ctePS_" + str(parameterID) + " AS ps ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.order <> ld.ftpperiod WHEN MATCHED THEN UPDATE SET ld.ftpperiod = ps.order + 1 ")
        
        spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_deleteplev2_" + str(parameterID) + "")
        
        spark.sql("CREATE TABLE  "+dbschema+".temp_deleteplev2_" + str(parameterID) + " AS SELECT LegacyPeriod,PeriodType from "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " where Period=LegacyPeriod")

        spark.sql("MERGE INTO "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " AS pl USING "+dbschema+".temp_deleteplev2_" + str(parameterID) + " AS tbm ON tbm.periodtype = pl.periodtype AND tbm.legacyperiod = pl.legacyperiod WHEN MATCHED THEN DELETE")
        
        spark.sql("MERGE INTO "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " AS tbm USING "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " AS pl ON tbm.periodtype = pl.periodtype AND tbm.legacyperiod = pl.legacyperiod WHEN MATCHED THEN DELETE")
        
        spark.sql("MERGE INTO "+dbschema+".temp_priorityseq_" + str(parameterID) + " AS ps USING "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld ON ps.periodtype = ld.periodtype AND ps.legacyperiod = ld.legacyperiod AND ps.order <> ld.ftpperiod - 1 WHEN MATCHED THEN DELETE")

        spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_bspriority_" + str(parameterID) + "")

        spark.sql("CREATE TABLE "+dbschema+".temp_bspriority_" + str(parameterID) + " AS SELECT PeriodType, legacyperiod, currentvalue, MAX(Period) AS Period, MAX(stdperiod) AS stdperiod FROM ( SELECT ld.PeriodType, ld.legacyperiod, ld.currentvalue, MAX(ps.Period) AS Period, MAX(ps.stdperiod) AS stdperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN "+dbschema+".temp_priorityseq_" + str(parameterID) + " ps ON ld.periodtype = ps.PeriodType AND ld.legacyperiod = ps.legacyperiod GROUP BY ld.PeriodType, ld.legacyperiod, ld.currentvalue UNION SELECT ld.PeriodType, ld.legacyperiod, ld.currentvalue, MAX(ps.Period) AS Period, MAX(ps.stdperiod) AS stdperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld JOIN "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " ps ON ld.periodtype = ps.PeriodType AND ld.legacyperiod = ps.legacyperiod GROUP BY ld.PeriodType, ld.legacyperiod, ld.currentvalue UNION SELECT PeriodType, legacyperiod, LEFT(legacyperiod, 6), MAX(Period), MAX(stdperiod) FROM "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " GROUP BY PeriodType, legacyperiod, legacyperiod ) a GROUP BY PeriodType, legacyperiod, currentvalue ")

        spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " AS ld USING "+dbschema+".temp_bspriority_" + str(parameterID) + " AS ps ON ld.periodtype = ps.PeriodType AND ld.legacyperiod = ps.legacyperiod WHEN MATCHED THEN UPDATE SET ld.LegacyPeriod = ps.Period, ld.CurrentValue = ps.stdperiod")

        spark.sql("MERGE INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " as Ys USING "+dbschema+".temp_legacysddates_" + str(parameterID) + " as ld ON ys.periodtype = ld.periodtype and financialyear = '"+str(Extendedcycleyear)+"' WHEN MATCHED THEN DELETE")

        spark.sql("MERGE INTO "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " AS tb  USING ( SELECT * FROM ( SELECT * FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " WHERE LEFT(Period, 6) <> LEFT(LegacyPeriod, 6) ) a  WHERE NOT EXISTS ( SELECT LegacyPeriod  FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " b  WHERE a.LegacyPeriod = Period ) ) del  ON tb.LegacyPeriod = del.LegacyPeriod  WHEN MATCHED THEN DELETE" )

        if countofcomp == 0 or countofcomp  == 1 :
            spark.sql("INSERT INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " SELECT PeriodTYpe, LEFT(legacyperiod, 6) stdperiod, Period FROM "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " WHERE Periodtype LIKE '%YTD' OR Periodtype LIKE '%Year' UNION SELECT ld.PeriodTYpe, LEFT(ld.legacyperiod, 6), Period FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " ps JOIN "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON ps.periodtype = ld.periodtype WHERE (ld.Periodtype LIKE '%YTD' OR ld.Periodtype LIKE '%Year') AND financialyear = '"+str(Extendedcycleyear)+"' UNION SELECT ld.PeriodTYpe, LEFT(pl.legacyperiod, 6), Period FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " pl JOIN "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON pl.periodtype = ld.periodtype WHERE (ld.Periodtype LIKE '%YTD' OR ld.Periodtype LIKE '%Year') AND financialyear = '"+str(Extendedcycleyear)+"'")
        
        if countofcomp > 1 :
            spark.sql("INSERT INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " SELECT periodtype, pcurrentvalue, legacyperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE periodtype NOT IN ('currentPeriod', 'PreviousPeriod') AND financialyear = '"+str(Extendedcycleyear)+"' EXCEPT SELECT periodtype, pcurrentvalue, legacyperiod FROM ( SELECT periodtype, pcurrentvalue, legacyperiod, financialyear FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE LEFT(legacyperiod, 6) <> pcurrentvalue ) a WHERE NOT EXISTS ( SELECT legacyPeriod FROM "+dbschema+".temp_tblegacymatchPeriod_" + str(parameterID) + " b WHERE a.legacyPeriod = Period AND a.periodtype = PeriodType ) AND financialyear = '"+str(Extendedcycleyear)+"' UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_priorityseq_" + str(parameterID) + " UNION SELECT PeriodTYpe, LEFT(legacyperiod, 6), Period FROM "+dbschema+".temp_prioritylevel2_" + str(parameterID) + " UNION SELECT periodtype, pcurrentvalue, Period FROM ( SELECT periodtype, LEFT(legacyperiod, 6) AS pcurrentvalue, Period FROM "+dbschema+".temp_tblegacymatchperiod_" + str(parameterID) + " EXCEPT SELECT periodtype, pcurrentvalue, legacyperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE financialyear = '"+str(Extendedcycleyear)+"' ) a ")

In [0]:
def GetLegacyAdjustmentByCategoryDrilldown(AccountingStandard,EntityID,PeriodValue,Currency,Category):
    
    queryString="DROP TABLE IF EXISTS {vSQLDB}.temp_templegacy_dd_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_tmpLegacywithoutendperiod_dd_" + str(parameterID) + ";DROP TABLE IF EXISTS {vSQLDB}.temp_tmplegacywithoutNA_dd_" + str(parameterID) + ";" 
    
    queryString=queryString.replace("{vSQLDB}",f"{dbschema}")
    
    sqlwarehouse_queryExecutor(queryString)

    CategoryName=""
    GivenYear=""

    spark.sql("CREATE TABLE "+dbschema+".temp_tmpLegacywithoutendperiod_dd_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported')  SELECT  LegacyAdjustmentID, AccountingStdID_fk, Period, EntityID_fk,  JournalNumber, JournalDate, UserGLCode, DebitCredit, CurrencyID_fk, Amount, JournalType, Narration, CreatedDate, CreatedBy, ModifiedDate, ModifiedBy, BUID_fk, IsImported, CategoryID, IsMappedGLCode, EndPeriod, NotApplicable,"+alldimcols+" FROM  "+dbschema+".temp_legacyadjustment_" + str(parameterID) + " WHERE  BUID_fk=0 AND CategoryID ="+str(Category)+" AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandard )+" AND Period <= "+str(PeriodValue )+" AND IFNULL(EndPeriod, 999999-99) > "+str(PeriodValue )+" ")

    spark.sql("CREATE TABLE "+dbschema+".temp_tmplegacywithoutNA_dd_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') SELECT *,case when Period = value then 0 when (NotApplicable is not null and NotApplicable !='' and value='') then 0 else 1 end IsmappedGLUpdatedCode FROM (SELECT *,explode(split(NVL(NotApplicable,''), ',')) as value  FROM "+dbschema+".temp_tmpLegacywithoutendperiod_dd_" + str(parameterID) + ") as t")

    spark.sql("CREATE TABLE "+dbschema+".temp_tempLegacy_dd_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') SELECT DISTINCT LegacyAdjustmentID, AccountingStdID_fk, Period, EntityID_fk,  JournalNumber, JournalDate, UserGLCode, DebitCredit, CurrencyID_fk, Amount, JournalType, Narration, CreatedDate, CreatedBy, ModifiedDate, ModifiedBy, BUID_fk, IsImported, CategoryID, IsMappedGLCode, EndPeriod, NotApplicable,"+alldimcols+" FROM "+dbschema+".temp_tmplegacywithoutNA_dd_" + str(parameterID) + "  where IsmappedGLUpdatedCode=1 and period not in (select distinct value from "+dbschema+".temp_tmplegacywithoutNA_dd_" + str(parameterID) + " )")

    spark.sql("INSERT INTO "+dbschema+".legacyadjustmentsdrilldown( AccountingStdID_fk, Period, EntityID_fk,  JournalNumber, JournalDate, UserGLCode, DebitCredit, CurrencyID_fk, Amount, JournalType, Narration, CreatedDate, CreatedBy, ModifiedDate, ModifiedBy, BUID_fk, IsImported, CategoryID, IsMappedGLCode, EndPeriod, NotApplicable, "+alldimcols+" ) SELECT  AccountingStdID_fk, Period, EntityID_fk,  JournalNumber, JournalDate, UserGLCode, DebitCredit, CurrencyID_fk, Amount, JournalType, Narration, CreatedDate, CreatedBy, ModifiedDate, ModifiedBy, BUID_fk, IsImported, CategoryID, IsMappedGLCode, EndPeriod, NotApplicable,"+alldimcols+" FROM "+dbschema+".temp_tempLegacy_dd_" + str(parameterID) + "")

In [0]:
spark.sql("MERGE INTO "+dbschema+".temp_ytddates_" + str(parameterID) + " t USING (SELECT row_number() over(partition by  PeriodType,CurrentValue,FTPValue order by CurrentValue )seq,* FROM "+dbschema+".temp_ytddates_" + str(parameterID) + " ) s ON t.PeriodType=s.PeriodType and t.CurrentValue=s.CurrentValue and  t.FTPValue=s.FTPValue and s.seq=2 WHEN MATCHED THEN DELETE")

spark.sql("CREATE TABLE "+dbschema+".temp_ladjjfinal_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') AS  SELECT *, CAST('' AS VARCHAR(50)) AS convertedperiod, CAST('' AS VARCHAR(100)) AS periodtype FROM "+dbschema+".legacyadjustmentsdrilldown WHERE 1=0")

spark.sql("CREATE TABLE "+dbschema+".temp_castingsdadjustment1_" + str(parameterID) + " TBLPROPERTIES ('delta.feature.allowColumnDefaults' = 'supported') AS  SELECT * FROM "+dbschema+".CastingAdjustmentTable WHERE 1=0")

spark.sql("TRUNCATE TABLE "+dbschema+".legacyadjustmentsdrilldown")

legacyadj_df = spark.sql("SELECT id,legacyperiod,financialyear FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + "").collect()

if len(legacyadj_df) > 0 :

    for row in legacyadj_df:

        spark.sql("MERGE INTO "+dbschema+".temp_legacysddates_" + str(parameterID) + " USING ( SELECT categoryid FROM v_categorydropdown WHERE categoryname = '"+str(CategorySDName)+"' AND accountingstandardid = "+str(AccountingStandardID)+" AND entityid = "+str(EntityID)+" AND Year = '"+str(row["financialyear"])+"' ) AS src ON (1=1) and id="+str(row["id"])+" WHEN MATCHED THEN UPDATE SET categoryid = COALESCE(src.categoryid, 0)")

        temp_legacysddates_df=spark.sql("SELECT legacyperiod,categoryid,castingperiod FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE id="+str(row["id"])+" ").collect()

        if len(temp_legacysddates_df)>0 :

            for row_1 in temp_legacysddates_df : 
                GetLegacyAdjustmentByCategoryDrilldown(AccountingStandardID,EntityID,row_1["legacyperiod"],SDCurrencyid,row_1["categoryid"])
                
                spark.sql("INSERT INTO "+dbschema+".temp_castingsdadjustment1_" + str(parameterID) + " SELECT * FROM "+dbschema+".temp_castingsdadjustment_" + str(parameterID) + " WHERE categoryid = '"+str(row_1["categoryid"])+"' AND fk_entityid = "+str(EntityID)+" AND fk_accountingstandardid = "+str(AccountingStandardID)+" AND roundoff = '"+str(Roundoff)+"' AND amountsin = '"+str(AmountsIn)+"' AND periodvalue = '"+str(row_1["castingperiod"])+"' AND roundtypelevel = '"+str(RoundTypeLevel)+"'")
                



In [0]:
queryString="DROP TABLE IF EXISTS  {vSQLDB}.temp_tbfinal_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_tbfinalytd_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_disclosures_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_disclosuresytd_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_tbfinaldd_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_tbfinalddytd_" + str(parameterID) + ";" 

queryString=queryString.replace("{vSQLDB}",f"{dbschema}")

sqlwarehouse_queryExecutor(queryString)

queryString="CREATE TABLE {vSQLDB}.temp_tbfinal_" + str(parameterID) + " ( PeriodType VARCHAR(50), Period VARCHAR(50), userglcode VARCHAR(50), currency INT, closingbalance DECIMAL(28, 8),"+alldimcols_dt+");CREATE TABLE {vSQLDB}.temp_tbfinalytd_" + str(parameterID) + " ( PeriodType VARCHAR(50), Period VARCHAR(50), userglcode VARCHAR(50), currency INT, closingbalance DECIMAL(28, 8), "+alldimcols_dt+" );CREATE TABLE {vSQLDB}.temp_tbfinaldd_" + str(parameterID) + " ( PeriodType VARCHAR(50), Period VARCHAR(50), userglcode VARCHAR(50), usergldescription VARCHAR(200), currency INT, closingbalance DECIMAL(28, 8), Source VARCHAR(50), "+alldimcols_dt+" );CREATE TABLE {vSQLDB}.temp_tbfinalddytd_" + str(parameterID) + " ( PeriodType VARCHAR(50), Period VARCHAR(50), userglcode VARCHAR(50), usergldescription VARCHAR(200), currency INT, closingbalance DECIMAL(28, 8), Source VARCHAR(50), "+alldimcols_dt+" );CREATE TABLE {vSQLDB}.temp_disclosures_" + str(parameterID) + " ( PeriodType VARCHAR(50), Period VARCHAR(50), entityid INT, ultimateparent VARCHAR(50), categoryid INT, member VARCHAR(200), accounttype VARCHAR(50), amount DECIMAL(28, 8), currencyid INT, column27 VARCHAR(100), AccountSubType VARCHAR(500), "+alldimcols_dt+" );CREATE TABLE {vSQLDB}.temp_disclosuresytd_" + str(parameterID) + " ( PeriodType VARCHAR(50), Period VARCHAR(50), entityid INT, ultimateparent VARCHAR(50), categoryid INT, member VARCHAR(200), accounttype VARCHAR(50), amount DECIMAL(28, 8), currencyid INT, column27 VARCHAR(100), AccountSubType VARCHAR(500), "+alldimcols_dt+" );DROP TABLE IF EXISTS {vSQLDB}.temp_legacyDatesLite_" + str(parameterID) + ";CREATE TABLE {vSQLDB}.temp_legacyDatesLite_" + str(parameterID) + " ( LegacyPeriod VARCHAR(20), PeriodType VARCHAR(500) );"

queryString=queryString.replace("{vSQLDB}",f"{dbschema}")

sqlwarehouse_queryExecutor(queryString)

spark.sql("INSERT INTO "+dbschema+".temp_legacyDatesLite_" + str(parameterID) + " SELECT DISTINCT LegacyPeriod,PeriodType FROM  "+dbschema+".temp_legacysddates_" + str(parameterID) + " ")


In [0]:
########## Below Need To Cross Verify with Current System Changes ################
temp_tbfinal_df=spark.sql("SELECT * FROM "+dbschema+".temp_tbfinal_" + str(parameterID) + " LIMIT 1 ").collect()

if len(temp_tbfinal_df)<=0 :
    ########## Below Need To Cross Verify with Current System Changes ################
    spark.sql("INSERT INTO "+dbschema+".temp_tbfinal_" + str(parameterID) + " SELECT a.PeriodType,a.Period, a.userglcode, currency, Sum(closingbalance) as closingbalance, "+alldimcols+" FROM ( SELECT PeriodType,LEFT(Period,6) as Period, userglcode, closingbalance, CurrencyID_fk AS currency, "+alldimcols+" FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " vt INNER JOIN "+dbschema+".temp_legacyDatesLite_" + str(parameterID) + " ld ON vt.Period = ld.legacyperiod AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" UNION ALL SELECT PeriodType,LEFT(Period,6) as Period, userglcode, amount, CurrencyID_fk, "+alldimcols+" FROM "+dbschema+".adjustments vt INNER JOIN "+dbschema+".temp_legacyDatesLite_" + str(parameterID) + " ld ON vt.Period = ld.legacyperiod AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" AND IsUserGLCodeAdjustment=1 INNER JOIN "+dbschema+".temp_tempcategory_" + str(parameterID) + " tc ON tc.categoryid = vt.CategoryID_fk UNION ALL SELECT PeriodType,LEFT(convertedperiod, 6) AS Period, userglcode, amount, CurrencyID_fk, "+alldimcols+" FROM "+dbschema+".temp_ladjjfinal_" + str(parameterID) + " WHERE ismappedglcode = 1 ) A GROUP BY PeriodType,Period, userglcode, currency, "+alldimcols+"")
    
    spark.sql("INSERT INTO "+dbschema+".temp_tbfinalytd_" + str(parameterID) + " SELECT a.PeriodType,a.Period, a.userglcode, currency, Sum(closingbalance) as closingbalance, "+alldimcols+" FROM ( SELECT PeriodType,LEFT(Period,6) as Period, userglcode, Sum(closingbalance) as closingbalance, CurrencyID_fk AS currency,"+alldimcols+" FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " vt INNER JOIN "+dbschema+".temp_ytddates_" + str(parameterID) + " yt ON vt.Period = yt.FTPValue AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" GROUP BY PeriodType,Period,userglcode,CurrencyID_fk, "+alldimcols+"  UNION  SELECT PeriodType,LEFT(Period,6) as Period, userglcode, sum(amount) as amount, CurrencyID_fk, "+alldimcols+" FROM "+dbschema+".adjustments vt INNER JOIN "+dbschema+".temp_ytddates_" + str(parameterID) + " yt ON vt.Period = yt.FTPValue AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" AND IsUserGLCodeAdjustment=1 INNER JOIN "+dbschema+".temp_tempcategory_" + str(parameterID) + " tc ON tc.categoryid = vt.CategoryID_fk GROUP BY PeriodType,Period,userglcode,CurrencyID_fk, "+alldimcols+" UNION  SELECT PeriodType,LEFT(convertedperiod, 6) AS Period, userglcode, amount, CurrencyID_fk, "+alldimcols+" FROM "+dbschema+".temp_ladjjfinal_" + str(parameterID) + " WHERE ismappedglcode = 1 ) A GROUP BY PeriodType,Period, userglcode, currency, "+alldimcols+"")


    #Select bulk upload disclsore data  if category not there else categorywise data


    spark.sql("INSERT INTO "+dbschema+".temp_disclosures_" + str(parameterID) + " ( PeriodType,Period, entityid, ultimateparent, categoryid, member, accounttype, amount, currencyid, column27, AccountSubType, "+alldimcols+" ) SELECT PeriodType,LEFT(Period,6) as Period, EntityID_fk, vt.ultimateparent, CategoryID_fk, vt.member, accounttype, amount, CurrencyID_fk, '' column27, vt.AccountSubType, "+alldimcols+" FROM "+dbschema+".adjustments vt INNER JOIN "+dbschema+".temp_allperiodFTPscoa_" + str(parameterID) + " sc on vt.Member=sc.Member and vt.AccountSubType = sc.AccountSubType INNER JOIN "+dbschema+".temp_legacyDatesLite_" + str(parameterID) + " ld ON vt.Period = ld.LegacyPeriod AND EntityID_fk ="+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" WHERE lower(AdjustmentDisclosureType)='disclosures' AND IsActive = 1")

    spark.sql("INSERT INTO "+dbschema+".temp_disclosuresytd_" + str(parameterID) + " ( PeriodType,Period, entityid, ultimateparent, categoryid, member, accounttype, amount, currencyid, column27, AccountSubType, "+alldimcols+" ) SELECT PeriodType,LEFT(Period,6) as Period, EntityID_fk, vt.ultimateparent, CategoryID_fk, vt.member, accounttype, sum(amount) as amount, CurrencyID_fk, '' column27, vt.AccountSubType, "+alldimcols+" FROM "+dbschema+".adjustments vt INNER JOIN "+dbschema+".temp_allperiodFTPscoa_" + str(parameterID) + " sc on vt.Member=sc.Member and vt.AccountSubType = sc.AccountSubType INNER JOIN "+dbschema+".temp_ytddates_" + str(parameterID) + " yt ON vt.Period = yt.FTPValue AND EntityID_fk ="+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" WHERE lower(AdjustmentDisclosureType)='disclosures' AND IsActive = 1 GROUP BY PeriodType,Period,EntityID_fk, vt.ultimateparent, CategoryID_fk, vt.member, accounttype, CurrencyID_fk, vt.AccountSubType, "+alldimcols+"")

    spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_updatechildftp_" + str(parameterID) + "")
    
    spark.sql("CREATE TABLE "+dbschema+".temp_updatechildftp_" + str(parameterID) + " as SELECT sc.member,sc.path FROM "+dbschema+".temp_disclosures_" + str(parameterID) + " d INNER JOIN "+dbschema+".temp_allperiodftpscoa_" + str(parameterID) + " sc ON d.member=sc.member  and d.AccountSubType = sc.AccountSubType WHERE sc.column27 = 'FTP' ")
    
    spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_updatechildftp_01_" + str(parameterID) + "")
    spark.sql("DROP TABLE IF EXISTS "+dbschema+".temp_updatechildftp_" + str(parameterID) + "")
    
    spark.sql("CREATE TABLE "+dbschema+".temp_updatechildftp_01_" + str(parameterID) + " as SELECT sc.member,sc.path FROM "+dbschema+".temp_disclosures_" + str(parameterID) + " d INNER JOIN "+dbschema+".temp_allperiodftpscoa_" + str(parameterID) + " sc ON d.member=sc.member  and d.AccountSubType = sc.AccountSubType WHERE sc.column27 = 'FTP' ")
    
    spark.sql("CREATE TABLE "+dbschema+".temp_updatechildftp_" + str(parameterID) + " SELECT sc.member,sc.Path,'FTP' column27 FROM "+dbschema+".temp_updatechildftp_01_" + str(parameterID) + " up INNER JOIN  "+dbschema+".temp_allperiodftpscoa_" + str(parameterID) + " sc ON Charindex(up.path,sc.path)>0")
    
    spark.sql("MERGE INTO "+dbschema+".temp_disclosures_" + str(parameterID) + " d USING "+dbschema+".temp_updatechildftp_" + str(parameterID) + " sc ON d.member=sc.member WHEN MATCHED THEN UPDATE SET d.column27=COALESCE(sc.column27,'')")
    
    spark.sql("MERGE INTO "+dbschema+".temp_disclosuresytd_" + str(parameterID) + " d USING "+dbschema+".temp_updatechildftp_" + str(parameterID) + " sc ON d.member=sc.member WHEN MATCHED THEN UPDATE SET d.column27=COALESCE(sc.column27,'')")
    
    spark.sql("INSERT INTO "+dbschema+".temp_tbfinaldd_" + str(parameterID) + "(PeriodType,Period,userglcode,usergldescription,currency,closingbalance,source,"+alldimcols+") SELECT PeriodType,a.Period, a.userglcode, a.usergldescription, currency, Sum(closingbalance) AS closingbalance, a.source, "+alldimcols+" FROM ( SELECT PeriodType,left(Period,6) as Period, userglcode, '' as usergldescription, closingbalance, CurrencyID_fk AS currency, 'TB' source, "+alldimcols+" FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " vt INNER JOIN "+dbschema+".temp_legacyDatesLite_" + str(parameterID) + " ld ON vt.Period = ld.LegacyPeriod AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" UNION  SELECT PeriodType,left(Period,6) as Period, userglcode, '' as usergldescription, amount, CurrencyID_fk, 'Adjustments' source, "+alldimcols+" FROM "+dbschema+".temp_standaloneadj_" + str(parameterID) + " vt INNER JOIN "+dbschema+".temp_legacyDatesLite_" + str(parameterID) + " ld ON vt.Period = ld.LegacyPeriod AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" AND IsUserGLCodeAdjustment=1 INNER JOIN "+dbschema+".temp_tempcategory_" + str(parameterID) + " tc ON tc.categoryid = vt.CategoryID_fk UNION  SELECT PeriodType,LEFT(convertedperiod, 6) AS Period, userglcode, '' as usergldescription, amount, CurrencyID_fk, 'Adjustments' source, "+alldimcols+" FROM "+dbschema+".temp_ladjjfinal_" + str(parameterID) + " WHERE ismappedglcode = 1 ) a GROUP BY PeriodType,Period, userglcode, usergldescription, currency, Source,"+alldimcols+"")
    
    spark.sql("INSERT INTO "+dbschema+".temp_tbfinalddytd_" + str(parameterID) + "(PeriodType,Period,userglcode,usergldescription,currency,closingbalance,source,"+alldimcols+") SELECT PeriodType,a.Period, a.userglcode, a.usergldescription, currency, Sum(closingbalance) AS closingbalance, a.source, "+alldimcols+" FROM ( SELECT PeriodType,left(Period,6) as Period, userglcode, '' as usergldescription, Sum(closingbalance) as closingbalance, CurrencyID_fk AS currency, 'TB' source, "+alldimcols+" FROM "+dbschema+".temp_trialbalance_" + str(parameterID) + " vt INNER JOIN "+dbschema+".temp_ytddates_" + str(parameterID) + " yt ON vt.Period = yt.FTPValue AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+"  GROUP BY PeriodType,Period, userglcode, CurrencyID_fk, "+alldimcols+" UNION  SELECT PeriodType,left(Period,6) as Period, userglcode, '' as usergldescription,sum(amount) as amount, CurrencyID_fk, 'Adjustments' source, "+alldimcols+" FROM "+dbschema+".temp_standaloneadj_" + str(parameterID) + " vt INNER JOIN "+dbschema+".temp_ytddates_" + str(parameterID) + " yt ON vt.Period = yt.FTPValue AND EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" AND IsUserGLCodeAdjustment=1 INNER JOIN "+dbschema+".temp_tempcategory_" + str(parameterID) + " tc ON tc.categoryid = vt.CategoryID_fk GROUP BY PeriodType,Period,userglcode,CurrencyID_fk, "+alldimcols+" UNION  SELECT PeriodType,LEFT(convertedperiod, 6) AS Period, userglcode, '' as usergldescription, amount, CurrencyID_fk, 'Adjustments' source, "+alldimcols+" FROM "+dbschema+".temp_ladjjfinal_" + str(parameterID) + " WHERE ismappedglcode = 1 ) a GROUP BY PeriodType,Period, userglcode, usergldescription, currency, Source, "+alldimcols+"")


In [0]:
queryString="DROP TABLE IF EXISTS  {vSQLDB}.temp_mappedsdtbytd_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_mappedsdtbftp_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_mappedtbddftp_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_mappedtbddytd_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_mappedsdtbytddaterange_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_mappedtbddytddaterange_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_LegacyDatesPeriodTypeLite_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_LegacyDatesNotPeriodTypeLite_" + str(parameterID) + ";DROP TABLE IF EXISTS  {vSQLDB}.temp_ISNoData_" + str(parameterID) + ";" 

queryString=queryString.replace("{vSQLDB}",f"{dbschema}")

sqlwarehouse_queryExecutor(queryString)

spark.sql("MERGE INTO "+dbschema+".temp_tbfinal_" + str(parameterID) + " mb USING  "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON  mb.period = ld.currentvalue AND mb.PeriodType = ld.periodtype WHEN MATCHED THEN UPDATE SET Period = ld.pcurrentvalue")

spark.sql("MERGE INTO "+dbschema+".temp_tbfinalytd_" + str(parameterID) + " mb USING  "+dbschema+".temp_ytddates_" + str(parameterID) + " yt ON   mb.period = LEFT(yt.ftpvalue, 6) AND    mb.PeriodType = yt.periodtype WHEN MATCHED THEN UPDATE SET  Period = yt.currentvalue")

spark.sql("MERGE INTO "+dbschema+".temp_tbfinaldd_" + str(parameterID) + " mb USING  "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON mb.period = ld.currentvalue AND    mb.PeriodType = ld.periodtype WHEN MATCHED THEN UPDATE SET  Period = ld.pcurrentvalue")

spark.sql("MERGE INTO "+dbschema+".temp_tbfinalddytd_" + str(parameterID) + " mb USING  "+dbschema+".temp_ytddates_" + str(parameterID) + " yt ON   mb.period = LEFT(yt.ftpvalue, 6) and  mb.PeriodType = yt.periodtype WHEN MATCHED THEN UPDATE SET  Period = yt.currentvalue")

spark.sql("MERGE INTO "+dbschema+".temp_disclosures_" + str(parameterID) + " mb USING  "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON  mb.period = ld.currentvalue AND   mb.PeriodType = ld.periodtype  WHEN MATCHED THEN UPDATE SET  Period = ld.pcurrentvalue")

spark.sql("MERGE INTO "+dbschema+".temp_disclosuresytd_" + str(parameterID) + " mb USING  "+dbschema+".temp_ytddates_" + str(parameterID) + " yt ON mb.period = LEFT(yt.ftpvalue, 6) AND    mb.PeriodType = yt.periodtype WHEN MATCHED THEN UPDATE SET  Period = yt.currentvalue")

spark.sql("CREATE TABLE "+dbschema+".temp_LegacyDatesPeriodTypeLite_" + str(parameterID) + "(pcurrentvalue VARCHAR(50),PeriodType VARCHAR(500))")

spark.sql("INSERT INTO "+dbschema+".temp_LegacyDatesPeriodTypeLite_" + str(parameterID) + " SELECT DISTINCT pcurrentvalue,PeriodType FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE PeriodType LIKE '%YTD' OR PeriodType LIKE '%Year'")

spark.sql("CREATE TABLE "+dbschema+".temp_LegacyDatesNotPeriodTypeLite_" + str(parameterID) + "(pcurrentvalue VARCHAR(50),PeriodType VARCHAR(500))")

spark.sql("INSERT INTO "+dbschema+".temp_LegacyDatesNotPeriodTypeLite_" + str(parameterID) + " SELECT DISTINCT pcurrentvalue,PeriodType FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE PeriodType NOT LIKE '%YTD' AND PeriodType NOT LIKE '%Year'")

### YTD ###

spark.sql("CREATE TABLE "+dbschema+".temp_mappedsdtbytd_" + str(parameterID) + " AS SELECT tb.PeriodType, tb.Period AS Period, tb.userglcode AS UserGLCode, tb.currency, Sum(closingbalance) closingbalance, mt.categoryid AS categoryid, mt.eyglcode AS member, 'Y' ytdtype, "+alldimcols+"  FROM "+dbschema+".temp_tbfinalytd_" + str(parameterID) + " TB INNER JOIN "+dbschema+".temp_scoamappingtable_" + str(parameterID) + " MT ON tb.userglcode = mt.userglcode  JOIN "+dbschema+".temp_LegacyDatesPeriodTypeLite_" + str(parameterID) + " ld ON ld.pcurrentvalue = tb.Period AND ld.PeriodType = tb.PeriodType WHERE ultimateparent = 'IS_GROUP' GROUP BY tb.PeriodType, tb.Period, tb.userglcode, tb.currency, mt.categoryid, mt.eyglcode, "+alldimcols+" UNION  SELECT tb.PeriodType, tb.Period AS Period, tb.userglcode AS UserGLCode, tb.currency, Sum(closingbalance), mt.categoryid AS categoryid, mt.eyglcode AS member, 'Y' ytdtype, "+alldimcols+" FROM "+dbschema+".temp_tbfinal_" + str(parameterID) + " TB INNER JOIN "+dbschema+".temp_scoamappingtable_" + str(parameterID) + " MT ON tb.userglcode = mt.userglcode JOIN "+dbschema+".temp_LegacyDatesPeriodTypeLite_" + str(parameterID) + " ld ON ld.pcurrentvalue = tb.Period AND ld.PeriodType = tb.PeriodType WHERE ultimateparent = 'BS_GROUP' GROUP BY tb.PeriodType, tb.Period, tb.userglcode, tb.currency, mt.categoryid, mt.eyglcode, "+alldimcols+" UNION  SELECT PeriodType, Period, NULL AS UserGLCode, currencyid AS currency, Sum(amount) AS closingbalance, categoryid, member, 'Y', "+alldimcols+" FROM "+dbschema+".temp_disclosuresytd_" + str(parameterID) + " WHERE ultimateparent NOT IN ('BS_GROUP', 'IS_GROUP') AND column27 = 'FTP'  GROUP BY PeriodType, Period, currencyid, categoryid, member, ultimateparent, column27, "+alldimcols+" UNION  SELECT disc.PeriodType, Period, NULL AS UserGLCode, currencyid AS currency, Sum(amount) AS closingbalance, categoryid, member, 'Y', "+alldimcols+" FROM "+dbschema+".temp_disclosures_" + str(parameterID) + " disc JOIN "+dbschema+".temp_LegacyDatesPeriodTypeLite_" + str(parameterID) + " ld ON ld.pcurrentvalue = disc.Period AND ld.PeriodType = disc.PeriodType WHERE ultimateparent NOT IN ('BS_GROUP', 'IS_GROUP') AND column27 <> 'FTP' GROUP BY disc.PeriodType, Period, currencyid, categoryid, member, ultimateparent, column27, "+alldimcols+" ")

### FTP ###

spark.sql("CREATE TABLE "+dbschema+".temp_mappedsdtbftp_" + str(parameterID) + " AS SELECT tb.PeriodType, tb.Period AS Period, tb.userglcode AS UserGLCode, tb.currency, closingbalance, mt.categoryid AS categoryid, mt.eyglcode AS member, 'N' ytdtype, "+alldimcols+"  FROM "+dbschema+".temp_tbfinal_" + str(parameterID) + " TB INNER JOIN "+dbschema+".temp_scoamappingtable_" + str(parameterID) + " MT ON tb.userglcode = mt.userglcode JOIN "+dbschema+".temp_LegacyDatesNotPeriodTypeLite_" + str(parameterID) + " ld ON ld.pcurrentvalue = tb.Period AND ld.PeriodType = tb.PeriodType  UNION ALL SELECT d.PeriodType, Period, NULL AS UserGLCode, currencyid AS currency, amount AS closingbalance, categoryid, member, 'N', "+alldimcols+" FROM "+dbschema+".temp_disclosures_" + str(parameterID) + " d JOIN "+dbschema+".temp_LegacyDatesNotPeriodTypeLite_" + str(parameterID) + " ld ON ld.pcurrentvalue = d.Period AND ld.PeriodType = d.PeriodType  UNION ALL SELECT ld.PeriodType, LEFT(Period, 6) Period, NULL AS UserGLCode, currencyid_fk currencyid, amount AS closingbalance, tc.categoryid, '' member, 'N', "+alldimcols+" FROM "+dbschema+".temp_standaloneadj_" + str(parameterID) + " sd JOIN ( SELECT DISTINCT LegacyPeriod, PeriodType FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " where lower(periodtype) not like '%ytd' and   lower(periodtype) not like '%year' )ld ON sd.Period=ld.LegacyPeriod INNER JOIN "+dbschema+".temp_tempcategory_" + str(parameterID) + " tc ON tc.categoryid = sd.CategoryID_fk WHERE EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+"  AND IsUserGLCodeAdjustment = 1 UNION ALL SELECT PeriodType, LEFT(convertedperiod, 6) AS Period, NULL userglcode, currencyid_fk currencyid, amount AS closingbalance, categoryid, '' member, 'N', "+alldimcols+" FROM "+dbschema+".temp_ladjjfinal_" + str(parameterID) + " WHERE ismappedglcode=1")

### drilldown YTD & FTP data ###
### YTD ###

spark.sql("CREATE TABLE "+dbschema+".temp_mappedtbddytd_" + str(parameterID) + " SELECT tb.PeriodType, tb.Period AS Period, tb.userglcode AS UserGLCode, tb.UserGLDescription AS UserGLDescription, Sum(closingbalance) closingbalance, mt.eyglcode AS member, tb.Source, 'Y' ytdtype, "+alldimcols+" FROM "+dbschema+".temp_tbfinalddytd_" + str(parameterID) + " TB INNER JOIN "+dbschema+".temp_scoamappingtable_" + str(parameterID) + " MT ON tb.userglcode = mt.userglcode JOIN ( SELECT pcurrentvalue, PeriodType FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE PeriodType LIKE '%YTD' OR PeriodType LIKE '%Year')ld ON ld.pcurrentvalue = tb.Period AND ld.PeriodType = tb.PeriodType WHERE ultimateparent = 'IS_GROUP' GROUP BY tb.PeriodType, tb.Period, tb.userglcode, tb.UserGLDescription, mt.eyglcode, tb.Source, "+alldimcols+" UNION SELECT tb.PeriodType, tb.Period AS Period, tb.userglcode AS UserGLCode, tb.UserGLDescription AS UserGLDescription, Sum(closingbalance) closingbalance, mt.eyglcode AS member, tb.Source, 'Y' ytdtype, "+alldimcols+" FROM "+dbschema+".temp_tbfinaldd_" + str(parameterID) + " TB INNER JOIN "+dbschema+".temp_scoamappingtable_" + str(parameterID) + " MT ON tb.userglcode = mt.userglcode JOIN ( SELECT pcurrentvalue, PeriodType FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE PeriodType LIKE '%YTD' OR PeriodType LIKE '%Year')ld ON ld.pcurrentvalue = tb.Period AND ld.PeriodType = tb.PeriodType WHERE ultimateparent = 'BS_GROUP' GROUP BY tb.PeriodType, tb.Period, tb.userglcode, tb.UserGLDescription, mt.eyglcode, tb.Source, "+alldimcols+" UNION SELECT PeriodType, Period, NULL AS UserGLCode, NULL AS UserGLDescription, Sum(amount) AS closingbalance, member, 'Disclosures', 'Y', "+alldimcols+" FROM "+dbschema+".temp_disclosuresytd_" + str(parameterID) + " WHERE ultimateparent NOT IN ('BS_GROUP', 'IS_GROUP') AND column27 = 'FTP' GROUP BY PeriodType, Period, member, ultimateparent, column27, "+alldimcols+" UNION SELECT disc.PeriodType, Period, NULL AS UserGLCode, NULL AS UserGLDescription, Sum(amount) AS closingbalance, member, 'Disclosures', 'Y', "+alldimcols+" FROM "+dbschema+".temp_disclosures_" + str(parameterID) + " disc JOIN ( SELECT pcurrentvalue, PeriodType FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + " WHERE PeriodType LIKE '%YTD' OR PeriodType LIKE '%Year')ld ON ld.pcurrentvalue = disc.Period AND ld.PeriodType = disc.PeriodType WHERE ultimateparent NOT IN ('BS_GROUP', 'IS_GROUP') AND column27 <> 'FTP' GROUP BY disc.PeriodType, Period, member, ultimateparent, column27, "+alldimcols+" ")

### FTP ###

spark.sql(" CREATE TABLE "+dbschema+".temp_mappedtbddftp_" + str(parameterID) + " SELECT tb.PeriodType, tb.Period AS Period, tb.userglcode AS UserGLCode, tb.UserGLDescription AS UserGLDescription, closingbalance, mt.eyglcode AS member, tb.Source, 'N' ytdtype, "+alldimcols+" FROM "+dbschema+".temp_tbfinaldd_" + str(parameterID) + " TB INNER JOIN "+dbschema+".temp_scoamappingtable_" + str(parameterID) + " MT ON tb.userglcode = mt.userglcode JOIN "+dbschema+".temp_LegacyDatesNotPeriodTypeLite_" + str(parameterID) + " ld ON ld.pcurrentvalue = tb.Period AND ld.PeriodType = tb.PeriodType UNION SELECT d.PeriodType, Period, NULL AS UserGLCode, NULL AS UserGLDescription, amount AS closingbalance, member, 'Disclosures', 'N', "+alldimcols+" FROM "+dbschema+".temp_disclosures_" + str(parameterID) + " d JOIN "+dbschema+".temp_LegacyDatesNotPeriodTypeLite_" + str(parameterID) + " ld ON ld.pcurrentvalue = d.Period AND ld.PeriodType = d.PeriodType UNION SELECT PeriodType, LEFT(Period, 6) Period, NULL AS UserGLCode, NULL AS UserGLDescription, amount AS closingbalance, '' member, 'Adjustments', 'N', "+alldimcols+" FROM "+dbschema+".temp_standaloneadj_" + str(parameterID) + " sd JOIN "+dbschema+".temp_legacysddates_" + str(parameterID) + " ld ON sd.Period=ld.LegacyPeriod and (lower(ld.periodtype) not like '%ytd' and   lower(ld.periodtype) not like '%year') INNER JOIN "+dbschema+".temp_tempcategory_" + str(parameterID) + " tc ON tc.categoryid = sd.CategoryID_fk WHERE EntityID_fk = "+str(EntityID)+" AND AccountingStdID_fk = "+str(AccountingStandardID)+" AND IsUserGLCodeAdjustment = 1 UNION SELECT PeriodType, LEFT(convertedperiod, 6) AS Period, NULL userglcode, NULL AS UserGLDescription, amount AS closingbalance, '' member, 'Adjustments', 'N', "+alldimcols+" FROM "+dbschema+".temp_ladjjfinal_" + str(parameterID) + " WHERE ismappedglcode=1")

if PeriodValueInYHQM == 0 :
    spark.sql("UPDATE "+dbschema+".temp_mappedsdtbftp_" + str(parameterID) + " SET periodtype = REPLACE(periodtype, '_YTD', '')")
    spark.sql("UPDATE "+dbschema+".temp_mappedsdtbytd_" + str(parameterID) + " SET periodtype = REPLACE(periodtype, '_YTD', '')")
    spark.sql("UPDATE "+dbschema+".temp_mappedtbddftp_" + str(parameterID) + " SET periodtype = REPLACE(periodtype, '_YTD', '')")
    spark.sql("UPDATE "+dbschema+".temp_mappedtbddytd_" + str(parameterID) + " SET periodtype = REPLACE(periodtype, '_YTD', '')")
    spark.sql("UPDATE "+dbschema+".temp_legacysddates_" + str(parameterID) + " SET periodtype = REPLACE(periodtype, '_YTD', '')")

### IS Period having prev period data ###
spark.sql("CREATE TABLE "+dbschema+".temp_ISNoData_" + str(parameterID) + " AS select lg.Periodtype,lg.legacyperiod from (select periodtype ,legacyperiod from "+dbschema+".temp_legacysddates_" + str(parameterID) + " where pcurrentvalue<>currentvalue)lg left join "+dbschema+".temp_ytddates_" + str(parameterID) + " yt on yt.FTPValue=lg.legacyperiod and yt.PeriodType=lg.periodtype where yt.Periodtype is null")

spark.sql("MERGE INTO "+dbschema+".temp_mappedsdtbftp_" + str(parameterID) + " as t USING ( SELECT mf.PeriodType FROM "+dbschema+".temp_mappedsdtbftp_" + str(parameterID) + " mf JOIN "+dbschema+".temp_SCOAMappingTable_" + str(parameterID) + " sc ON sc.EYGLCode = mf.member JOIN "+dbschema+".temp_ISNoData_" + str(parameterID) + " nd ON mf.PeriodType = nd.periodtype WHERE sc.UltimateParent = 'IS_GROUP' ) AS s ON t.PeriodType = s.PeriodType WHEN MATCHED THEN DELETE")

spark.sql("MERGE INTO "+dbschema+".temp_mappedtbddftp_" + str(parameterID) + " as t USING ( SELECT mf.PeriodType FROM "+dbschema+".temp_mappedtbddftp_" + str(parameterID) + " mf JOIN "+dbschema+".temp_SCOAMappingTable_" + str(parameterID) + " sc ON sc.EYGLCode = mf.member JOIN "+dbschema+".temp_ISNoData_" + str(parameterID) + " nd ON mf.PeriodType = nd.periodtype WHERE sc.UltimateParent = 'IS_GROUP' ) AS s ON t.PeriodType = s.PeriodType WHEN MATCHED THEN DELETE")


In [0]:

spark.sql("DROP TABLE IF EXISTS "+dbschema+"."+temp_staging_tbl+" ")
spark.sql("DROP TABLE IF EXISTS "+dbschema+"."+temp_drilldown_tbl+" ")

spark.sql("CREATE TABLE  "+dbschema+"."+temp_staging_tbl+" ( id INT DEFAULT NULL, PeriodType VARCHAR(50) DEFAULT NULL, Period VARCHAR(50) DEFAULT NULL, userglcode VARCHAR(100) DEFAULT NULL, currency INT DEFAULT NULL, closingbalance DECIMAL(28, 8) DEFAULT NULL, categoryid INT DEFAULT NULL, member VARCHAR(1000) DEFAULT NULL, ytdtype CHAR(10) DEFAULT NULL, usergldescription VARCHAR(200) DEFAULT NULL, Source VARCHAR(50) DEFAULT NULL, Value VARCHAR(5) DEFAULT NULL, pcurrentvalue VARCHAR(50) DEFAULT NULL, legacyperiod VARCHAR(20) DEFAULT NULL, financialyear VARCHAR(10) DEFAULT NULL, openingperiodvalue VARCHAR(20) DEFAULT NULL, castingperiod VARCHAR(20) DEFAULT NULL, p VARCHAR(20) DEFAULT NULL, ppp VARCHAR(20) DEFAULT NULL, py VARCHAR(20) DEFAULT NULL, pypp VARCHAR(20) DEFAULT NULL, ppy VARCHAR(20) DEFAULT NULL, p3y VARCHAR(20) DEFAULT NULL, p4y VARCHAR(20) DEFAULT NULL, CastingAdjustmentsID BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1), Fk_AccountingStandardID INT, Fk_ValidationID INT DEFAULT NULL, PeriodValue VARCHAR(30), Fk_EntityID INT, EntityName VARCHAR(50), JournalNumber INT, JournalDate DATE DEFAULT NULL, StandardGLCode VARCHAR(100) DEFAULT NULL, StandardGLDescription VARCHAR(250) DEFAULT NULL, DebitCredit VARCHAR(50) DEFAULT NULL, Fk_CurrencyID INT, Amount DECIMAL(18, 4) DEFAULT NULL, JournalType VARCHAR(50) DEFAULT NULL, Narration VARCHAR(250) DEFAULT NULL, Fk_BusinessUnitID INT DEFAULT NULL, Roundoff INT DEFAULT NULL, AmountsIn VARCHAR(30) DEFAULT NULL, ReportType VARCHAR(60) DEFAULT NULL, CreatedDate DATE DEFAULT NULL, CreatedBy VARCHAR(250) DEFAULT NULL, ModifiedDate DATE DEFAULT NULL, ModifiedBy VARCHAR(250) DEFAULT NULL, IsUserGLCode INT DEFAULT NULL, RoundTypeLevel VARCHAR(20) DEFAULT NULL, typeofdata INT, "+alldimcols_dt+" ) TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported') ")

spark.sql("CREATE TABLE  "+dbschema+"."+temp_drilldown_tbl+" ( id INT DEFAULT NULL, PeriodType VARCHAR(50) DEFAULT NULL, Period VARCHAR(50) DEFAULT NULL, userglcode VARCHAR(100) DEFAULT NULL, currency INT DEFAULT NULL, closingbalance DECIMAL(28, 8) DEFAULT NULL, categoryid INT DEFAULT NULL, member VARCHAR(1000) DEFAULT NULL, ytdtype CHAR(10) DEFAULT NULL, usergldescription VARCHAR(200) DEFAULT NULL, Source VARCHAR(50) DEFAULT NULL, Value VARCHAR(5) DEFAULT NULL, pcurrentvalue VARCHAR(50) DEFAULT NULL, legacyperiod VARCHAR(20) DEFAULT NULL, financialyear VARCHAR(10) DEFAULT NULL, openingperiodvalue VARCHAR(20) DEFAULT NULL, castingperiod VARCHAR(20) DEFAULT NULL, p VARCHAR(20) DEFAULT NULL, ppp VARCHAR(20) DEFAULT NULL, py VARCHAR(20) DEFAULT NULL, pypp VARCHAR(20) DEFAULT NULL, ppy VARCHAR(20) DEFAULT NULL, p3y VARCHAR(20) DEFAULT NULL, p4y VARCHAR(20) DEFAULT NULL, CastingAdjustmentsID BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1), Fk_AccountingStandardID INT, Fk_ValidationID INT DEFAULT NULL, PeriodValue VARCHAR(30), Fk_EntityID INT, EntityName VARCHAR(50), JournalNumber INT, JournalDate DATE DEFAULT NULL, StandardGLCode VARCHAR(100) DEFAULT NULL, StandardGLDescription VARCHAR(250) DEFAULT NULL, DebitCredit VARCHAR(50) DEFAULT NULL, Fk_CurrencyID INT, Amount DECIMAL(18, 4) DEFAULT NULL, JournalType VARCHAR(50) DEFAULT NULL, Narration VARCHAR(250) DEFAULT NULL, Fk_BusinessUnitID INT DEFAULT NULL, Roundoff INT DEFAULT NULL, AmountsIn VARCHAR(30) DEFAULT NULL, ReportType VARCHAR(60) DEFAULT NULL, CreatedDate DATE DEFAULT NULL, CreatedBy VARCHAR(250) DEFAULT NULL, ModifiedDate DATE DEFAULT NULL, ModifiedBy VARCHAR(250) DEFAULT NULL, IsUserGLCode INT DEFAULT NULL, RoundTypeLevel VARCHAR(20) DEFAULT NULL, typeofdata INT, "+alldimcols_dt+" ) TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported') ")

spark.sql("INSERT INTO "+dbschema+"."+temp_staging_tbl+" ( typeofdata, id, PeriodType, Value, pcurrentvalue, legacyperiod, categoryid , financialyear, openingperiodvalue, castingperiod, p, ppp, py, pypp, ppy, p3y, p4y ) SELECT 1, id, PeriodType, Value, pcurrentvalue, legacyperiod, categoryid, financialyear, openingperiodvalue, castingperiod, p, ppp, py, pypp, ppy, p3y, p4y FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + "")

spark.sql("INSERT INTO "+dbschema+"."+temp_drilldown_tbl+" ( typeofdata, id, PeriodType, Value, pcurrentvalue, legacyperiod, categoryid , financialyear, openingperiodvalue, castingperiod, p, ppp, py, pypp, ppy, p3y, p4y ) SELECT 1, id, PeriodType, Value, pcurrentvalue, legacyperiod, categoryid, financialyear, openingperiodvalue, castingperiod, p, ppp, py, pypp, ppy, p3y, p4y FROM "+dbschema+".temp_legacysddates_" + str(parameterID) + "")

spark.sql("INSERT INTO "+dbschema+"."+temp_staging_tbl+" ( typeofdata,id,Fk_AccountingStandardID,Fk_ValidationID,PeriodValue,FinancialYear,Fk_EntityID ,EntityName,JournalNumber,JournalDate,UserGLCode,UserGLDescription,StandardGLCode,StandardGLDescription,DebitCredit,Fk_CurrencyID,Amount,JournalType,Narration,Fk_BusinessUnitID,CategoryID,Roundoff,AmountsIn,ReportType,CreatedDate,CreatedBy,ModifiedDate,ModifiedBy,IsUserGLCode,RoundTypeLevel,"+alldimcols+" ) SELECT DISTINCT 2,castingadjustmentsid,fk_accountingstandardid,fk_validationid,periodvalue,financialyear,fk_entityid,entityname,journalnumber,journaldate,userglcode,usergldescription,standardglcode,standardgldescription,debitcredit,fk_currencyid,amount,journaltype,narration,fk_businessunitid,categoryid,roundoff,amountsin,reporttype,createddate,createdby,modifieddate,modifiedby,isuserglcode,roundtypelevel,"+alldimcols+" FROM  "+dbschema+".temp_castingsdadjustment1_" + str(parameterID) + "")

spark.sql("INSERT INTO "+dbschema+"."+temp_drilldown_tbl+" ( typeofdata,id,Fk_AccountingStandardID,Fk_ValidationID,PeriodValue,FinancialYear,Fk_EntityID ,EntityName,JournalNumber,JournalDate,UserGLCode,UserGLDescription,StandardGLCode,StandardGLDescription,DebitCredit,Fk_CurrencyID,Amount,JournalType,Narration,Fk_BusinessUnitID,CategoryID,Roundoff,AmountsIn,ReportType,CreatedDate,CreatedBy,ModifiedDate,ModifiedBy,IsUserGLCode,RoundTypeLevel,"+alldimcols+" ) SELECT DISTINCT 2,castingadjustmentsid,fk_accountingstandardid,fk_validationid,periodvalue,financialyear,fk_entityid,entityname,journalnumber,journaldate,userglcode,usergldescription,standardglcode,standardgldescription,debitcredit,fk_currencyid,amount,journaltype,narration,fk_businessunitid,categoryid,roundoff,amountsin,reporttype,createddate,createdby,modifieddate,modifiedby,isuserglcode,roundtypelevel,"+alldimcols+" FROM  "+dbschema+".temp_castingsdadjustment1_" + str(parameterID) + "")

df_tbl_count_01=spark.sql("SELECT * FROM system.information_schema.tables where lower(concat(table_catalog,'.',table_schema))=lower('"+dbschema+"') and table_name in ('temp_mappedsdtbftp_" + str(parameterID) + "','temp_mappedsdtbytd_" + str(parameterID) + "')")
df_tbl_count_02=spark.sql("SELECT * FROM system.information_schema.tables where lower(concat(table_catalog,'.',table_schema))=lower('"+dbschema+"') and table_name in ('temp_mappedtbddftp_" + str(parameterID) + "','temp_mappedtbddytd_" + str(parameterID) + "')")

if df_tbl_count_01.count() == 2 :
    spark.sql("INSERT INTO "+dbschema+"."+temp_staging_tbl+" ( typeofdata, PeriodType, Period, userglcode, currency, closingbalance, categoryid, member, ytdtype, "+alldimcols+" ) SELECT 0, PeriodType, Period, userglcode, currency, closingbalance, categoryid, member, ytdtype,"+alldimcols+" FROM "+dbschema+".temp_mappedsdtbftp_" + str(parameterID) + " UNION SELECT 0, PeriodType, Period, userglcode, currency, closingbalance, categoryid, member, ytdtype, "+alldimcols+" FROM "+dbschema+".temp_mappedsdtbytd_" + str(parameterID) + " ")

if df_tbl_count_02.count() == 2 :
    spark.sql("INSERT INTO "+dbschema+"."+temp_drilldown_tbl+" ( typeofdata, PeriodType, Period, userglcode, usergldescription, closingbalance, member, source, ytdtype, "+alldimcols+" ) SELECT 0, PeriodType, Period, userglcode, usergldescription, closingbalance, member, source, ytdtype,"+alldimcols+" FROM "+dbschema+".temp_mappedtbddftp_" + str(parameterID) + " UNION SELECT 0, PeriodType, Period, userglcode, usergldescription, closingbalance, member, source, ytdtype, "+alldimcols+" FROM "+dbschema+".temp_mappedtbddytd_" + str(parameterID) + "")